In [ ]:
#Start Here for Making Flux Size Graphs

import os
os.chdir('analysis/multi_freq_from_archive')
#format beamsize number
def format_number(number):
        # Case 1: If the number is less than 1 (e.g., 0.004567)
        if number < 0:
                # Format the positive part of the number and prepend the negative sign
                return "-" + format_number(abs(number))
        if number < 1:
                # Convert the number to a string with high precision
                num_str = f"{number:.16g}"

                # Identify the leading zeros and decimal point
                leading_part = []
                for char in num_str:
                        if char == '0' or char == '.':
                                leading_part.append(char)
                        else:
                                break

                # Remove leading zeros and the decimal point for significant digits
                significant_digits = "".join(char for char in num_str if char.isdigit() and char != "0")

                # Extract the first two non-zero digits
                if len(significant_digits) >= 2:
                        first_two = significant_digits[:2]
                elif len(significant_digits) == 1:
                        first_two = significant_digits + "0"  # Pad with zero if only one significant digit exists
                else:
                        first_two = "00"  # Handle edge case like 0

                # Combine the leading zeros and the first two non-zero digits
                return "".join(leading_part) + first_two

        # Case 2: If the number is greater than or equal to 1
        else:
                # Format to two decimal places
                return f"{number:.2f}"

In [ ]:
#Create Spectral Plots
import matplotlib.pyplot as pylab
import matplotlib.colors
import numpy as np
import shutil
import os
import matplotlib.cm as cmx
import math
from matplotlib.ticker import MultipleLocator
import copy

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

msize=10

path = f'SpectraGraphs'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'
freqs=[]
fluxs=[]
beams=[]

with open(source_text, 'r') as infile:
    jfin=-1
    for line in infile:
        if len(line)<2:
            continue
        jfin=jfin+1 

grouped_data = {}

with open(source_text, 'r') as infile:
    j=-1
    for line in infile:
        if line=='\n':
            continue
        
        line=eval(line)
        source=line[0]
        j=j+1
        
        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            continue

        if source not in grouped_data:
            grouped_data[source] = {
                'sfreqs': [], 'sfluxs': [], 'sbeams': [], 'sdates': [],
                'sfreqs2': [], 'sfluxs2': [], 'sbeams2': [], 'sdates2': [],
            }

        if j==0:
            minfreq=freq
            maxfreq=freq
            minflux=flux
            maxflux=flux
            minbeam=beam
            maxbeam=beam
        if j!=0:
            if freq>maxfreq:
                maxfreq=freq
            if freq<minfreq:
                minfreq=freq
            if flux>maxflux:
                maxflux=flux
            if flux<minflux:
                minflux=flux
            if beam>maxbeam:
                maxbeam=beam
            if beam<minbeam:
                minbeam=beam

minfreqp=round_down_to_half_integer(minfreq)
maxfreqp=round_up_to_half_integer(maxfreq)

minfluxp=round_down_to_half_integer(minflux)
maxfluxp=round_up_to_half_integer(maxflux)

minbeamp=round_down_to_half_integer(minbeam)
maxbeamp=round_up_to_half_integer(maxbeam)

interval = maxfreqp - minfreqp
ten_percent = interval * 0.05
minfreq = minfreqp - ten_percent
maxfreq = maxfreqp + ten_percent

interval = maxfluxp - minfluxp
ten_percent = interval * 0.05
minflux = minfluxp - ten_percent
maxflux = maxfluxp + ten_percent

interval = maxbeamp - minbeamp
ten_percent = interval * 0.05
minbeam = minbeamp - ten_percent
maxbeam = maxbeamp + ten_percent


x_tick_positions=np.arange(minfreqp, maxfreqp + 0.5 , 0.5)  # 6 evenly spaced ticks
x_tick_labels = [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5, 0.5)  # 6 evenly spaced ticks
y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

stick_positions = np.arange(minbeamp, maxbeamp + 1, 1)  # 6 evenly spaced ticks
stick_labels = [f"{pos:.2f}" for pos in stick_positions]  # Convert log to linear

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'

sourcedone=[]
sources=[]

oldsources=[]
oldsource='NA'

with open(source_text, 'r') as infile:
    j=-1
    j2=-1
    for line in infile:
        j2=j2+1 
        if len(line)<2:
            continue
        line=eval(line)

        source=line[0]
        breaker=1
        j=j+1

        if j==0:
            sbeamsizes=[]
            sdates=[]
            sfluxs=[]
            sfreqs=[]
            sbeamsizes2=[]
            sdates2=[]
            sfluxs2=[]
            sfreqs2=[]

            
        breaker=0

        sources.append(source)
        sources=list(set(sources))

        if len(sources)-len(oldsources)!=0 and j!=0:
            source=oldsource

            for isource in sourcedone:
                if source==isource:
                    breaker=1
            if breaker==1:
                continue
            sourcedone.append(source)

            fig, ax = pylab.subplots()
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs/{source}'):
                os.makedirs(f'SpectraGraphs/{source}')
            if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
                os.remove(f'SpectraGraphs/{source}/{nicename}')
            #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
            #pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]

        if j2==jfin-1:
            if line=='\n':
                continue
 
            fig, ax = pylab.subplots()
            cm2 = pylab.get_cmap('jet')   
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source}',fontsize=20)
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs/{source}'):
                os.makedirs(f'SpectraGraphs/{source}')
            if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
                os.remove(f'SpectraGraphs/{source}/{nicename}')
            #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
            #pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


        oldsource=source

        flux_upperb=0
        if len(str(line[3]).split('*'))>1:
            flux_upperb=1

        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        odate=line[4]
        if odate!='NA':
            odate=odate.split('/')
            date=float(odate[0]) + float(odate[1]) / 12 + float(odate[2]) / 365
        else:
            date='NA'

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            continue

        if flux_upperb==0:
            sbeamsizes.append(beam)
            sfluxs.append(flux)
            sfreqs.append(freq)
            sdates.append(date)

        if flux_upperb==1:
            sbeamsizes2.append(beam)
            sfluxs2.append(flux)
            sfreqs2.append(freq)
            sdates2.append(date)

        oldsources=copy.deepcopy(sources)


In [ ]:
# functino to Group frequencies dynamically
def group_by_frequency(freqs, fluxes, beams, dates, tolerance):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    current_group_freqs, current_group_fluxes, current_group_beams, current_group_dates = [], [], [], []

    sorted_data = sorted(zip(freqs, fluxes, beams, dates), key=lambda x: x[0])
    for freq, flux, beam, date in sorted_data:
        if not current_group_freqs or abs(freq - current_group_freqs[0]) <= tolerance:
            current_group_freqs.append(freq)
            current_group_fluxes.append(flux)
            current_group_beams.append(beam)
            current_group_dates.append(date)
        else:
            grouped_freqs.append(current_group_freqs)
            grouped_fluxes.append(current_group_fluxes)
            grouped_beams.append(current_group_beams)
            grouped_dates.append(current_group_dates)
            current_group_freqs = [freq]
            current_group_fluxes = [flux]
            current_group_beams = [beam]
            current_group_dates = [date]

    if current_group_freqs:
        grouped_freqs.append(current_group_freqs)
        grouped_fluxes.append(current_group_fluxes)
        grouped_beams.append(current_group_beams)
        grouped_dates.append(current_group_dates)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Define frequency ranges for VLA and ALMA bands
vla_bands = [
    (0.0,0.054) ,     #for Rounding 
    (0.054, 0.086),  # 4 Band
    (0.230, 0.470),  # P Band
    (1, 2),          # L Band
    (2, 4),          # S Band
    (4, 8),          # C Band
    (8, 12),         # X Band
    (12, 18),        # Ku Band
    (18, 26.5),      # K Band
    (26.5, 40),      # Ka Band
    (40, 50),        # Q Band
]

alma_bands = [
    (31.3, 45),   # Band 1
    (67, 90),     # Band 2
    (84, 116),    # Band 3
    (125, 163),   # Band 4
    (163, 211),   # Band 5
    (211, 275),   # Band 6
    (275, 373),   # Band 7
    (385, 500),   # Band 8
    (602, 720),   # Band 9
    (787, 950),   # Band 10
]

# Combine all boundaries and create continuous intervals
all_boundaries = sorted(set([freq for band in vla_bands + alma_bands for freq in band]))
non_overlapping_intervals = [(all_boundaries[i], all_boundaries[i + 1]) for i in range(len(all_boundaries) - 1)]

# Remove gaps by ensuring no holes
continuous_intervals = []
for i, (start, end) in enumerate(non_overlapping_intervals):
    if i > 0 and start > continuous_intervals[-1][1]:
        # Fill the gap between the previous interval and the current one
        continuous_intervals.append((continuous_intervals[-1][1], start))
    continuous_intervals.append((start, end))

# Function to check if a frequency is in any interval
def is_frequency_in_intervals(frequency, intervals):
    for start, end in intervals:
        if start <= frequency < end:  # Check if frequency is in the interval
            return True, (start, end)
    return False, None

# Print all intervals for verification
print("\nAll continuous intervals:")
for i, interval in enumerate(continuous_intervals):
    print(f"Interval {i + 1}: {interval[0]}–{interval[1]} GHz")

def group_by_band(freqs, fluxes, beams, dates, intervals):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    interval_mapping = {interval: ([], [], [], []) for interval in intervals}

    for freq, flux, beam, date in zip(freqs, fluxes, beams, dates):
        found, assigned_interval = is_frequency_in_intervals(freq, intervals)
        if found:
            interval_mapping[assigned_interval][0].append(freq)
            interval_mapping[assigned_interval][1].append(flux)
            interval_mapping[assigned_interval][2].append(beam)
            interval_mapping[assigned_interval][3].append(date)

    for interval, (freq_list, flux_list, beam_list, date_list) in interval_mapping.items():
        if freq_list:
            grouped_freqs.append(freq_list)
            grouped_fluxes.append(flux_list)
            grouped_beams.append(beam_list)
            grouped_dates.append(date_list)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Generate Statistical Spectral Plots
# Define tolerance for grouping frequencies (log space)
tolerance = .05

#define if it should be grouped dynamicall or by band
grouper='byband'
#grouper='bydynam'

# Generate consistent x-axis and y-axis tick positions
x_tick_positions = np.linspace(minfreq, maxfreq, num=6)  # 6 evenly spaced ticks
x_tick_labels = [f"{10**pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.linspace(minflux, maxflux, num=6)  # 6 evenly spaced ticks
y_tick_labels = [f"{10**pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

# Apply grouping to all sources
for source, data in grouped_data.items():
    if grouper=='bydynam':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_frequency(
            data['sfreqs'], data['sfluxs'], data['sbeams'],data['sdates'],tolerance
        )
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_frequency(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'],tolerance
        )

    if grouper=='byband':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_band(
            data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'], continuous_intervals
        )
        
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_band(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'], continuous_intervals
        )


# Initialize combined arrays
combined_freqs = []  # Combined frequencies
combined_fluxes = []  # Combined fluxes
combined_beams = []  # Combined beam sizes

for source, data in grouped_data.items():
    # Combine all frequency, flux, and beam data for the current source
    combined_freqs = data['sfreqs'] 
    combined_fluxes = data['sfluxs'] 
    combined_beams = data['sbeams'] 
    combined_dates = data['sdates'] 

    combined_freqs_2 = data['sfreqs2']
    combined_fluxes_2 = data['sfluxs2'] 
    combined_beams_2 = data['sbeams2'] 
    combined_dates_2 = data['sdates2'] 

    # Group the combined data dynamically for the current source
    if grouper=='bydynam':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_frequency(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, tolerance
        )

    if grouper=='byband':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_band(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, continuous_intervals
        )

    # Iterate over grouped combined data
    for freq_group, flux_group, beam_group in zip(grouped_freqs, grouped_fluxes, grouped_beams):
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        if group_size >= 4:
            pylab.boxplot(flux_group, positions=[central_freq],widths=0.1,
            showfliers=False)  # Customize outliers
        elif group_size == 3:
            # Vertical line with caps at min and max
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = np.median(flux_group)
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5,  # Length of caps
                color='k'
            )
            # Add a horizontal line at the median_flux
            pylab.hlines(
                y=median_flux,  # Position of the horizontal line
                xmin=central_freq - 0.05,  # Adjust as needed for length
                xmax=central_freq + 0.05, 
                color='r',  # Color of the line
                linewidth=1  # Thickness of the horizontal line
            )
        elif group_size == 2:
            # Vertical line with caps at min and max (no median)
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = (min_flux + max_flux) / 2  # Just for centering the line
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5, 
                color='k'
            )

        elif group_size == 1:
            pylab.scatter([central_freq], [flux_group[0]], c='b', marker='o', s=10)

    if combined_fluxes_2:  # Ensure '2' fluxes exist
        if not combined_fluxes or min(combined_fluxes_2) < min(combined_fluxes):
            # If main fluxes are empty OR '2' minimum is lower
            min_flux_2 = min(combined_fluxes_2)
            min_flux_2_index = combined_fluxes_2.index(min_flux_2)
            min_freq_2 = combined_freqs_2[min_flux_2_index]
            # Mark the lowest value with a downward triangle
            pylab.scatter([min_freq_2], [min_flux_2], c='black', marker='v', s=7, label='Min Flux from 2')


    # Set consistent x and y limits for all plots
    pylab.xlim(minfreq - 0.1, maxfreq + 0.1)  # Add slight padding
    pylab.ylim(minflux - 0.1, maxflux + 0.1)  # Add slight padding
    # Set consistent x-axis ticks and labels
    # Set consistent x-axis and y-axis ticks and labels
    pylab.xticks(x_tick_positions, x_tick_labels)
    pylab.yticks(y_tick_positions, y_tick_labels)

    # Add labels and title
    pylab.xlabel('Frequency [GHz]')
    pylab.ylabel(r'Flux [mJy][Beam]$^{-1}$')
    pylab.title(f'{source}')

    nicename=f'{source} statistical spectra.pdf'
    if not os.path.exists(f'SpectraGraphs/{source}'):
        os.makedirs(f'SpectraGraphs/{source}')
    if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
        os.remove(f'SpectraGraphs/{source}/{nicename}')
    #pylab.savefig(f'SpectraGraphs/{source}/{nicename}')
    #pylab.show()
    pylab.clf() 

In [ ]:
#make function to get distances

import os
from astroquery.ned import Ned
import ast  # For safely parsing text file data

import os
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

from astropy.coordinates import SkyCoord
import numpy as np
import astropy.units as u

import re

import requests
import csv
from scipy import constants
c=constants.c

def is_subsequence(small, large):
    it = iter(large)
    return all(char in it for char in small)

def decompose_string(s):
    parts = re.findall(r"[A-Za-z]+|\d+", s)  # Finds all letter and number sequences
    return parts if parts else [s]

# Input file with source names and coordinates
txtnamesandcoords = 'namesandcoords.txt'

# List of desired reference codes
wantrefs = ['2018ApJ...863..', '2023ApJ...948..', '2020ApJ...890..', '2020ApJ...891..']

def getdists(newnames):
    newrefs=[]
    if not os.path.exists('distancetable.csv'):
        url = 'https://ned.ipac.caltech.edu/Archive/Distances/NED30.5.1-D-17.1.2-20200415.csv'
        response = requests.get(url)

        if response.status_code == 200:
            with open('distancetable.csv', "wb") as file:
                file.write(response.content)
        else:
            return(f"Failed to download. Status code: {response.status_code}")
    
    maserdist=[]
    otherdist=[]
    j1=-1
    j2=-1
    with open('distancetable.csv', "r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            textsource=row[3]
            if len(textsource)==0:
                continue
            textdistmm=row[4]
            textdistmmerr=row[5]

            textdistMpc=row[6]
            textmethod=row[7]
            refcode=row[8]


            textsource=textsource.replace(' ','')

            findred=0
            for i in newnames:
                checkers=[]
                for ii in i:
                    ii=ii.split('Galaxy')[0]
                    if is_subsequence(ii, textsource):
                        checkers.append(ii)
                if len(checkers)==len(i):
                    if row[10]:
                        findred=1
                        redshift=row[10]
                        Hubble=row[11]
                        newdist=c*float(redshift)/float(Hubble)*c
                    
                    if textdistmmerr:
                        if float(textdistmmerr)>0:
                            if 'maser' in textmethod.lower():
                                j1=j1+1
                                newrefs.append(refcode)
                                if j1==0:
                                        maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                        maserdisuncer=textdistmmerr
                                elif textdistmmerr<maserdisuncer:
                                    maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                            else:
                                j2=j2+1
                                if j2==0:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                    otherdisuncer=textdistmmerr
                                elif textdistmmerr<otherdisuncer:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]

    return(maserdist,otherdist,refcode,findred,newrefs)

# Function to process each source
def process_source(source_name, wantrefs, ra, dec,threshold, secondarynames):
  
    """Query NED for redshift data and filter based on references and uncertainties."""
    # Skip excluded sources

    # Query the redshift table from NED
    distances_table = Ned.get_table(source_name, table='redshifts')

    # Extract relevant columns
    redshifts = distances_table['Published Redshift']
    uncertainties = distances_table['Published Redshift Uncertainty']
    refcodes = distances_table['Refcode']

    # Combine into tuples
    entries = [
        (redshift, uncertainty, refcode)
        for redshift, uncertainty, refcode in zip(redshifts, uncertainties, refcodes)
    ]

    # Filter for entries with desired references
    desired_entries = [
        entry for entry in entries if any(ref in entry[2] for ref in wantrefs)
    ]

    entries_with_reference = []
    entries_without_reference = []

    for redshift, uncertainty, refcode in desired_entries:
        if any(ref in refcode for ref in wantrefs):  # Check if refcode contains a wanted reference
            entries_with_reference.append((redshift, uncertainty, refcode))
        else:
            entries_without_reference.append((redshift, uncertainty, refcode))


    reduced_entries = {}

    # Step 2a: Prioritize entries from `entries_with_reference`
    if entries_with_reference:
        for redshift, uncertainty, refcode in entries_with_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)

    if len(reduced_entries)==0:
        # Step 2b: If `entries_with_reference` is empty, use `entries_without_reference`
        for redshift, uncertainty, refcode in entries_without_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)



    # Print results based on the filtering
    redshift='NA'
    uncertainty='NA'

    newnames=[]
    newlist=decompose_string(secondarynames[0][0])

    newnames.append(newlist)
    if len(secondarynames[0][1])!=0:
        if secondarynames[0][1][0]!='NA':
            for i in secondarynames[0][1]:
                if not is_subsequence(secondarynames[0][0], i):
                    newlist=decompose_string(i)
                    newnames.append(newlist)

    maserdists, otherdists, inewrefs, findred, newrefs = getdists(newnames)

    if reduced_entries:
        for irefcode, (iredshift, iuncertainty) in reduced_entries.items():
            redshift=iredshift
            ynfindref='y'
            refcode=irefcode
            if iuncertainty>0:
                uncertainty=iuncertainty
            else:
                input(f'No uncertainty for chosen referencecode: {refcode} , {source}')
                uncertainty='NA'
    else:

        # If no desired references, and no maser distances, find valid entries with uncertainties
        valid_entries = [entry for entry in entries if entry[1] > 0]
        if valid_entries:
            # Select the entry with the lowest uncertainty
            lowest_uncertainty_entry = min(valid_entries, key=lambda x: x[1])
            lowest_redshift, lowest_uncertainty, reference_code = lowest_uncertainty_entry
            redshift=lowest_redshift
            uncertainty= lowest_uncertainty
            refcode=reference_code
            ynfindref='n'
        else:
            input(f"No valid redshift entries found for {source_name} (1)\n")
        if len(maserdists)!=0:
            z_observed = redshift
            if z_observed >= threshold:
                if len(maserdists)!=0 and findred==1:
                        input('new maser source found... adjust accordingly (1)')
        
    z_observed = redshift
    z_uncer=uncertainty

    if z_observed <= threshold:
        if len(maserdists)!=0:
            input('new maser source found... adjust accordingly (2)')
        if len(maserdists)==0:
            textdistmm=float(otherdists[1])
            textdistmmerr=float(otherdists[2])
            textdistMpc=float(otherdists[3])

            newdist=10**((textdistmm)/5-5)
            newdist_uncer=np.log(10)/5*newdist*textdistmmerr
            textmethod=otherdists[4]
            refcode=otherdists[5]

            return([newdist,newdist_uncer],textmethod,'NA',refcode,'DISTANCE',newrefs)

    # Calculate heliocentric velocity
    c = 299792.458  # Speed of light in km/s
    v_helio = z_observed * c
    v_helio_uncer = z_uncer * c

    #Hubble Constant
    #Pesce, APJ 891:L1
    H = 73.9
    H_uncer = 3
    dist=v_helio/H
    dist_uncer=np.sqrt((v_helio_uncer/H)**2 + (v_helio/H**2*H_uncer)**2)

    return([dist,dist_uncer],'NA',ynfindref,refcode,'REDSHIFT', newrefs)

In [ ]:
#Path on local computer to SMBH mass
from pathlib import Path
parent_dir = Path.cwd().parent
Greenepath=f'{parent_dir}/CountingBHS_and_Spectra/Greene_params/Greene_params.txt'

In [ ]:
#date format
from datetime import datetime, timedelta

def decimal_year_to_date(decimal_year):
    year = int(decimal_year)  # Extract the year
    fraction = decimal_year - year  # Extract the decimal part

    # Convert the decimal part to days (assuming 365 days in a year)
    days_elapsed = int(round(fraction * 365))  

    # Compute the actual date by adding days to January 1st of that year
    date = datetime(year, 1, 1) + timedelta(days=days_elapsed)

    # Format as YY.MM.DD
    return f"{date.year }/{date.month:02}/{date.day:02}"

In [ ]:
#get SMBH size
from scipy import constants
from astropy import constants as aconstants
from matplotlib.lines import Line2D

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
G=constants.gravitational_constant
pi=constants.pi

def linear_to_log_uncertainty(lin_x, lin_delta_x):
    return lin_delta_x / (lin_x * np.log(10))

def log_to_linear_uncertainty(log_x, log_x_uncertainty):
    return log_x_uncertainty * 10**log_x * np.log(10)

def getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc):
    mass=10**float(log_BH_mass_solar)*mass_sun
    mass_unc=mass*np.log(10)*float(log_BH_mass_solar_uncer)
    mass_fracerror=(mass_unc/mass)*100

    distance=distanceMPc*pc*10**6
    distance_uncer=distanceMPc_uncer*pc*10**6
    dist_fracerror=(distance_uncer/distance)*100
    
    if arcsectopc=='NA':
        m_d=mass/distance
        m_d_unc=np.sqrt((m_d/mass*mass_unc)**2+(m_d/distance*distance_uncer)**2)
        m_d_fracerror=(m_d_unc/m_d)*100
        diam_bhs_microarcsec_fracerror = m_d_fracerror

        diam_bhs_rad=2*np.sqrt(27)*G/(c**2)*m_d
        diam_bhs_deg=diam_bhs_rad*360/(2*pi)
        diam_bhs_arcsec=diam_bhs_deg*60*60
        diam_bhs_microarcsec=diam_bhs_arcsec*10**6

        diam_bhs_microarcsec_unc=diam_bhs_microarcsec/(m_d)*m_d_unc


        return(log_BH_mass_solar, log_BH_mass_solar_uncer, mass_fracerror, distanceMPc, distanceMPc_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

        
    diam_bhs=2*np.sqrt(27)*G/(c**2)*mass
    diam_bhs_unc = 2*np.sqrt(27)*G/(c**2)*mass_unc
    
    diam_bhspc=diam_bhs/pc
    diam_bhspc_unc=diam_bhs_unc/pc

    diam_bhs_arsec=diam_bhspc/arcsectopc
    diam_bhs_microarcsec=diam_bhs_arsec*10**6

    diam_bhs_arsec_unc=np.sqrt(
                                (diam_bhspc_unc/arcsectopc)**2
                                + ((-1)*diam_bhspc/arcsectopc**2*arcsectopc_unc)**2
                                )
    
    diam_bhs_microarcsec_unc=diam_bhs_arsec_unc*10**6
    diam_bhs_microarcsec_fracerror = (diam_bhs_microarcsec_unc/diam_bhs_microarcsec)*100

    return(mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

sagmass=6.6
sagmass_unc_up=.12
sagmass_unc_down=.07
sagdist=8.15*10**(-3)
sagdist_unc=.15*10**(-3)

distanceMPc=sagdist
distanceMPc_uncer=sagdist_unc

log_BH_mass_solar=sagmass
log_BH_mass_solar_uncerup=sagmass_unc_up
log_BH_mass_solar_uncerdown=sagmass_unc_down

mass, mass_uncup, mass_fracerrorup, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(sagmass,sagmass_unc_up,sagdist,sagdist_unc,'NA','NA')
mass, mass_uncdown, mass_fracerrordown, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(sagmass,sagmass_unc_down,sagdist,sagdist_unc,'NA','NA')

sagsize=diam_bhs_microarcsec
sagsizeunc_down=diam_bhs_microarcsec_uncdown
sagsizeunc_up=diam_bhs_microarcsec_uncup
logsagsizeunc_down=linear_to_log_uncertainty(sagsize, sagsizeunc_down)
logsagsizeunc_up=linear_to_log_uncertainty(sagsize, sagsizeunc_up)

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'SagA*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} ± {format_number(distanceMPc_uncer)} MPc ({format_number(dist_fracerror)}%)', linestyle='')
log_BH_mass_solar_uncerup=format_number(float(log_BH_mass_solar_uncerup))
log_BH_mass_solar_uncerdown=format_number(float(log_BH_mass_solar_uncerdown))
mass_fracerror=(mass_fracerrorup+mass_fracerrordown)/2
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} $^{{+{log_BH_mass_solar_uncerup}}}_{{-{log_BH_mass_solar_uncerdown}}}$ log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles1=[sourcehan,sizechan,masshan,disthan]

fig, ax = pylab.subplots()

m87mass=9.81
m87mass_unc=0.06
m87dist=16.8
m87dist_uncup=.8
m87dist_uncdown=.7

distanceMPc=m87dist
distanceMPc_uncerup=m87dist_uncup
distanceMPc_uncerdown=m87dist_uncdown

log_BH_mass_solar=m87mass
log_BH_mass_solar_uncer=m87mass_unc

mass, mass_unc, mass_fracerror, distance, distance_uncerup, dist_fracerrorup, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncup,'NA','NA')
mass, mass_unc, mass_fracerror, distance, distance_uncerdown, dist_fracerrordown, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncdown,'NA','NA')

m87size=diam_bhs_microarcsec
m87sizeunc_down=diam_bhs_microarcsec_uncdown
m87sizeunc_up=diam_bhs_microarcsec_uncup
logm87sizeunc_down=linear_to_log_uncertainty(m87size, m87sizeunc_down)
logm87sizeunc_up=linear_to_log_uncertainty(m87size, m87sizeunc_up)

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'M87*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
dist_fracerror=(distanceMPc_uncerup+distanceMPc_uncerdown)/2
distanceMPc_uncerup=format_number(float(distanceMPc_uncerup))
distanceMPc_uncerdown=format_number(float(distanceMPc_uncerdown))
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} $^{{+{distanceMPc_uncerup}}}_{{-{distanceMPc_uncerdown}}}$ MPc ({format_number(dist_fracerror)}%)', linestyle='')
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} ±{log_BH_mass_solar_uncerdown} log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles2=[sourcehan,sizechan,masshan,disthan]

legend1=ax.legend(handles=handles2, loc='upper center')   
ax.add_artist(legend1)
ax.legend(handles=handles1, loc='lower center')   
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks

# Remove tick labels
ax.set_xticklabels([])  # Remove x-axis labels
ax.set_yticklabels([])  # Remove y-axis labels
if not os.path.exists('Extras'):
    os.makedirs('Extras')
#pylab.savefig(f'Extras/SMBH of SagA* and M87')
pylab.clf() 


In [ ]:
#Extrapolate High, Low, and Chosen Observations
import matplotlib.pyplot as plt
import sys

write=True
#write=False

sigsmart=True

namesandfiles=[]

#set extrap to 1 just to extrapolate values for the chosen sources
extrap=1

if extrap==0:
    write=False

def round_to_sig_figs(sigsmart,x, n):
      if sigsmart==False:
          return(x,n)
      if x == 0:
            return 0
      else:
            digits = -int(math.floor(math.log10(abs(x)))) + (n - 1)
            return round(x, digits)
      
def round_to_sig_figs2(sigsmart,x, n):
      if sigsmart==False:
          return(x,n)
      if x == 0:
            return 0
      else:
            digits = -int(math.floor(math.log10(abs(x)))) + (n - 1)
            return round(x, digits)

def count_sig_figs(num_input):
			num_str = str(num_input).strip().replace(',', '')
			# Strip sign
			if num_str.startswith(('+', '-')):
						num_str = num_str[1:]

			# Handle scientific notation strings by expanding to a plain decimal string
			if 'e' in num_str.lower():
						mant, exp = num_str.lower().split('e', 1)
						exp = int(exp)

						# Separate mantissa integer/decimal
						if '.' in mant:
									int_part, dec_part = mant.split('.', 1)
						else:
									int_part, dec_part = mant, ''

						# Digits of mantissa with no dot
						digits = ''.join(ch for ch in (int_part + dec_part) if ch.isdigit())

						# Original decimal index within the digits string
						orig_idx = len(int_part)

						# New decimal index after shifting by exponent
						new_idx = orig_idx + exp

						if new_idx <= 0:
									integer = '0'
									decimal = '0' * (-new_idx) + digits
									num_str = integer + '.' + decimal
						elif new_idx >= len(digits):
									integer = digits + '0' * (new_idx - len(digits))
									num_str = integer  # no decimal part needed
						else:
									integer = digits[:new_idx]
									decimal = digits[new_idx:]
									num_str = integer + '.' + decimal

			# Now count sig figs on the plain decimal representation
			if '.' in num_str:
						integer, decimal = num_str.split('.', 1)
						sig_part = integer.lstrip('0') + ''.join(c for c in decimal if c.isdigit())
						return sum(c.isdigit() for c in sig_part)
			else:
						sig_part = num_str.lstrip('0').rstrip('0')
						return sum(c.isdigit() for c in sig_part)


def match_decimal_places(sigsmart,target, reference):
      if sigsmart==False:
          return f"{target}"
      # Count digits after decimal in the reference (as string)
      ref_str = str(reference)
      if '.' in ref_str:
            decimals = len(ref_str.split('.')[1])
      else:
            decimals = 0
      
      # Format the target to that number of decimal places
      return f"{target:.{decimals}f}"

sourcesofinterest=('4258','1194','3079','4945','Circinus','1068')

lim1=[200,400]

threshold=0.0015

get_freq_chosen=[]

def find_flux_with_smallest_beam(arrays,isource):
    # Find the index of the smallest beam size in the array
    # flux,fluxunc,beam,freq,date,telescope
    arrays = np.array(arrays)
    index_of_smallest_beam = np.argmin(arrays[:, 2].astype(float))
    flux = arrays[index_of_smallest_beam, 0]
    fluxunc = arrays[index_of_smallest_beam, 1]
    beam = arrays[index_of_smallest_beam, 2]
    freq = arrays[index_of_smallest_beam, 3]
    date = arrays[index_of_smallest_beam, 4].replace('_1','')
    tele = arrays[index_of_smallest_beam, 5]
    file = arrays[index_of_smallest_beam, 6].replace('\n','')
    return flux, fluxunc, beam, freq, date, tele, file

def find_fluxs_with_high_freq(arrays, logfreq_flux_with_smallest_beam):
    # Find the index of the smallest beam size in the array
    freq_chosen=10**float(logfreq_flux_with_smallest_beam)

    arrays = np.array(arrays)
    index_of_highest_freq = np.argmax(arrays[:, 3].astype(float))
    high_freq=10**float(arrays[index_of_highest_freq, 3])

    if high_freq==freq_chosen:
        return('NA')
    
    result_entries = []

    #flux = arrays[index_of_highest_freq, 0]
    #fluxunc = arrays[index_of_highest_freq, 1]
    #beam = arrays[index_of_highest_freq, 2]
    #freq = arrays[index_of_highest_freq, 3]
    #date = arrays[index_of_highest_freq, 4]
    #tele = arrays[index_of_highest_freq, 5]

    #result_entries.append((flux, fluxunc, beam, freq, date, tele))

    for i, entry in enumerate(arrays):
        ifreq = 10**float(entry[3])
        #if ifreq > freq_chosen:
        if (ifreq / freq_chosen) > 1.099:
            # Frequency is more than 10% 10\\% higher than the chosen frequency
            result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], entry[5]))

    return  result_entries

def find_fluxs_with_low_freq(arrays, logfreq_flux_with_smallest_beam):
    # Find the index of the smallest beam size in the array
    freq_chosen=10**float(logfreq_flux_with_smallest_beam)

    arrays = np.array(arrays)
    #these variables should be changed to represent lowest frequency... it was copy and pasted from find_fluxs_with_high_freq
    index_of_highest_freq = np.argmin(arrays[:, 3].astype(float))
    high_freq=10**float(arrays[index_of_highest_freq, 3])

    if high_freq==freq_chosen:
        return('NA')
    
    result_entries = []

    #flux = arrays[index_of_highest_freq, 0]
    #fluxunc = arrays[index_of_highest_freq, 1]
    #beam = arrays[index_of_highest_freq, 2]
    #freq = arrays[index_of_highest_freq, 3]
    #date = arrays[index_of_highest_freq, 4]
    #tele = arrays[index_of_highest_freq, 5]
    #result_entries.append((flux, fluxunc, beam, freq, date, tele))

    for i, entry in enumerate(arrays):
        ifreq = 10**float(entry[3])
        #if ifreq < freq_chosen:
        if (ifreq / freq_chosen) < 0.901:
            # Frequency is more than 10% 10\\% lower than the chosen frequency
            result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], entry[5]))

    return  result_entries

def dust_extrap_lowest(arrays, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam):

    freq_chosen=10**float(logfreq_flux_with_smallest_beam)
    beam_chosen=10**float(logbeam_flux_with_smallest_beam)

    result_entries = []

    # Find the index of the smallest beam size in the array
    for entry in arrays:
        ilogflux = float(entry[0])
        iflux = 10**float(entry[0])
        
        ilogfluxunc = entry[1]
 
        if ilogfluxunc!='NA':
            ifluxunc = log_to_linear_uncertainty(ilogflux, float(ilogfluxunc))
        else:
            ifluxunc = 'NA'

        ibeam = 10**float(entry[2])
        ifreq = 10**float(entry[3])

        iflux_extrap = iflux * (freq_chosen/ifreq)**4.5

        #dust emission is extended... not a point source
        iflux_extrap = iflux_extrap * (beam_chosen/ibeam)**2

        if ifluxunc!='NA':
            iflux_extrap_unc = ifluxunc * (freq_chosen/ifreq)**4.5
            iflux_extrap_unc = iflux_extrap_unc * (beam_chosen/ibeam)**2
            iflux_extrap_unc = linear_to_log_uncertainty(iflux_extrap,iflux_extrap_unc)
        else:
            iflux_extrap_unc = 'NA'

        iflux_extrap= np.log10(iflux_extrap)
        result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], iflux_extrap, iflux_extrap_unc, entry[5]))

    # Find the entry with the lowest extrapolated flux
    min_flux_entry = min(result_entries, key=lambda x: x[5])  # x[5] accesses the iflux_extrap
    max_flux_entry = max(result_entries, key=lambda x: x[5])

    # Return the details from the entry with the lowest extrapolated flux
    flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele = min_flux_entry  # Unpacking the details from the entry tuple
    flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2 = max_flux_entry  # Unpacking the details from the entry tuple
    return [flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele],[flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2]


def jet_extrap(arrays, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam):

    freq_chosen=10**float(logfreq_flux_with_smallest_beam)
    beam_chosen=10**float(logbeam_flux_with_smallest_beam)

    result_entries = []

    # Find the index of the smallest beam size in the array
    for entry in arrays:
        ilogflux = float(entry[0])
        iflux = 10**float(entry[0])
        
        ilogfluxunc = entry[1]
 
        if ilogfluxunc!='NA':
            ifluxunc = log_to_linear_uncertainty(ilogflux, float(ilogfluxunc))
        else:
            ifluxunc = 'NA'

        ibeam = 10**float(entry[2])
        ifreq = 10**float(entry[3])

        iflux_extrap = iflux * (freq_chosen/ifreq)**(-0.5)

        #compact jet is a point source... no beam correction
        #iflux_extrap = iflux_extrap * (beam_chosen/ibeam)**2

        if ifluxunc!='NA':
            iflux_extrap_unc = ifluxunc * (freq_chosen/ifreq)**(-0.5)
            iflux_extrap_unc = linear_to_log_uncertainty(iflux_extrap,iflux_extrap_unc)
        else:
            iflux_extrap_unc = 'NA'

        iflux_extrap= np.log10(iflux_extrap)
        result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], iflux_extrap, iflux_extrap_unc, entry[5]))

    # Find the entry with the lowest extrapolated flux
    min_flux_entry = min(result_entries, key=lambda x: x[5])  # x[1] accesses the iflux_extrap
    max_flux_entry = max(result_entries, key=lambda x: x[5])
    # Return the details from the entry with the lowest extrapolated flux
    flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele = min_flux_entry  # Unpacking the details from the entry tuple
    flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2 = max_flux_entry  # Unpacking the details from the entry tuple

    return [flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele],[flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2]


def linear_to_log_uncertainty(lin_x, lin_delta_x):
    return lin_delta_x / (lin_x * np.log(10))

def log_to_linear_uncertainty(log_x, log_x_uncertainty):
    return float(log_x_uncertainty) * 10**float(log_x) * np.log(10)

graph_array1=[]
graph_array2=[]

nograph=[]

                        #'flux'
                        #'frequency'         
                        #'beam size'
                        #'date'  

for isource, data in grouped_data.items():
    breaker=1
    if extrap==1:
        for i in sourcesofinterest:
            if i.lower() in isource.lower():
                breaker=0
        if breaker==1:
            continue

    contam_table=f'contam_{isource}.tex'

    with open(contam_table, 'w') as file:
        if write==True:
            original_stdout = sys.stdout
            sys.stdout = file

        # Main loop to read and process sources from the input file
        if os.path.exists(txtnamesandcoords):
            with open(txtnamesandcoords, 'r') as infile:
                for line in infile:

                    # Parse the line into a list of name-coordinate pairs
                    params = ast.literal_eval(line)
                    for inamecoords in params:                
                        source_name = inamecoords[0][0]
                        if not isource.lower() in source_name.lower():
                            continue
                        source=source_name
                        ra = inamecoords[1][1]
                        dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                        findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                        distanceMPc=findvals[0][0]
                        distanceMPc_uncer=findvals[0][1]
                        method=findvals[1]
                        ynfindref=findvals[2]
                        refcode=findvals[3]
                        red_depend=findvals[4]
        else:
            print(f"File {txtnamesandcoords} does not exist.")

        arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
        arcsectopc=arcsectokpc*1000

        arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
        arcsectopc_unc=arcsectokpc_error*1000

        arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

        distance='NA'
        with open(Greenepath, 'r') as infile:
            j=-1
            breaker=1
            for line in infile:
                j=j+1
                if j>20:
                    continue
                source_name=line.split(' ')[0]
                source_name=source_name.replace("−","-")
                if isource=='WISEAJ043703':
                    isource='J0437'
                if isource.lower() in source_name.lower():
                    log_BH_mass_solar=line.split(' ')[3]
                    log_BH_mass_solar_offset=line.split(' ')[5]
                    breaker=0
                if 'cgcg' in isource.lower():
                    log_BH_mass_solar=7.38
                    log_BH_mass_solar_offset=0.012
                    breaker=0
                if '4945' in isource.lower():
                    log_BH_mass_solar=6.15
                    log_BH_mass_solar_offset='NA'
                    breaker=0
            if breaker!=1:
                mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)
            else:
                print(f'{source} not found')
            

            fluxs1=[]
            fluxs2=[]
            fluxsuplim=[]
            with open(source_text, 'r') as infile:
                j=-1
                for line in infile:
                    if line=='\n':
                        continue
                    line=eval(line)
                    source=line[0]
                    if isource in source:
                        file=line[7]
                        telescope=line[1]
                        if telescope=='VLBA':
                            continue
                        date=line[4].replace('_1','')
                        snr=line[6].split('\n')[0]

                        freq=np.log10(float(line[2]))
                        flux=np.log10(float(str(line[3]).split('*')[0]))
                        checklow=len(str(line[3]).split('*'))
                        if snr!='NA':
                            linflux=float(line[3].split('*')[0])
                            linfluxunc=float(linflux)/float(snr)

                            fluxunc=linear_to_log_uncertainty(linflux,linfluxunc)
                        else:
                            fluxunc='NA'

                        if '-' not in line[5]:
                            beam=np.log10(float(line[5]))
                        else:
                            continue

                        #for extrap==0 we are looking for an obs to fix the coordinates for all the fits
                        #we look for an observations in the 200-400 GHz range that are a detection... then to 
                        #all other frequency ranges.  We look for the highest resolution that is a detection
                        if extrap==0 and checklow==1:
                            if 10**freq<lim1[1] and 10**freq>lim1[0]:
                                if date=='2021/09/03':
                                    #input(linflux)
                                    #input(linfluxunc)
                                    pass
                                fluxs1.append([flux,fluxunc,beam,freq,date,telescope,file])
                            fluxs2.append([flux,fluxunc,beam,freq,date,telescope,file])

                        #for extrap==1 we are extrapolating to the chosen frequency... which is the frequency
                        #of the obs of the highest resolution in the 200-400GHz range.  In this case we do not 
                        #care about lower limits. If the observation of highest resolution is a lower limit this
                        #is what is reported... furthermore extrapolations are upper limits so that we include 
                        #non detections for this as well
                        if extrap==1:
                            if 10**freq<lim1[1] and 10**freq>lim1[0]:
                                if date=='2021/09/03':
                                    #input(linflux)
                                    #input(linfluxunc)
                                    pass
                                fluxs1.append([flux,fluxunc,beam,freq,date,telescope,file])
                            fluxs2.append([flux,fluxunc,beam,freq,date,telescope,file])
            if fluxs1:      
                graph_array1.append(isource)
                flux_with_smallest_beam = find_flux_with_smallest_beam(fluxs1,isource)

                if flux_with_smallest_beam[1]!='NA':
                    sobsfluxerr=f'{log_to_linear_uncertainty(flux_with_smallest_beam[0],flux_with_smallest_beam[1]):.1f}'
                    if sobsfluxerr=='0.0':
                        sobsflux=f'{10**float(flux_with_smallest_beam[0]):.2f}'
                        sobsfluxerr=f'{log_to_linear_uncertainty(flux_with_smallest_beam[0],flux_with_smallest_beam[1]):.2f}'
                    else:
                        sobsflux=f'{10**float(flux_with_smallest_beam[0]):.1f}'
                else:
                    sobsfluxerr='NA'
                    sobsflux=f'{10**float(flux_with_smallest_beam[0]):.1f}'
                
                
                namesandfiles.append([isource,'chosenfile',flux_with_smallest_beam[6]])
                if extrap==0:
                    continue

                logbeam_flux_with_smallest_beam=flux_with_smallest_beam[2]
                logfreq_flux_with_smallest_beam=flux_with_smallest_beam[3]
                get_freq_chosen.append([isource,10**float(logfreq_flux_with_smallest_beam),10**float(logbeam_flux_with_smallest_beam)])

                graph_array1.append(flux_with_smallest_beam)
                log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
                log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
                if '4945' in isource:
                    log_diam_bhs_microarcsec_unc=0
                bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
                graph_array1.append(bhs)
                flux_high_freq_high_beam='NA'
                flux_low_freq_low_beam='NA'

                beamobs=10**float(flux_with_smallest_beam[2])
                beamobs=format_number(beamobs)

                beamhigh='NA'
                beamlow='NA'
                freqlow='NA'

                print(fr"""\begin{{deluxetable*}}{{lllllll}}\
\tablecaption{{Continuum Flux and Extrapolation to {10**float(flux_with_smallest_beam[3]):.0f} GHz for {isource}}}
\tablehead{{ 
\colhead{{Source}} & \colhead{{Extrapolation}} & \colhead{{Telescope}} & \colhead{{Flux[mJy[beam]$^{{-1}}]$}} & \colhead{{Frequency[GHz]}} & \colhead{{beamsize["]}} & \colhead{{Date[yyyy/mm/dd]}}
}}
\startdata""")
                if fluxs2:
                    fluxs_with_highest_freq = find_fluxs_with_high_freq(fluxs2,logfreq_flux_with_smallest_beam)
                    fluxs_with_lowest_freq = find_fluxs_with_low_freq(fluxs2,logfreq_flux_with_smallest_beam)
                    

                    obsflux_str = sobsflux            # e.g. "12.30" or "0.0450"
                    # Count sig figs from the string to preserve trailing zeros:
                    obs_sigs = count_sig_figs(obsflux_str)

                    if fluxs_with_highest_freq != 'NA':
                        flux_high_freq_high_beam, flux_high_freq_high_beam2 = dust_extrap_lowest(fluxs_with_highest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                        #low lim
                        beamhigh=10**float(flux_high_freq_high_beam[2])
                        beamhigh=format_number(beamhigh)

                        highflux=10**float(flux_high_freq_high_beam[5])
                        highfluxerr=log_to_linear_uncertainty(flux_high_freq_high_beam[5],flux_high_freq_high_beam[6])

                        highflux=round_to_sig_figs(sigsmart,highflux, obs_sigs)
                        highfluxerr=match_decimal_places(sigsmart,highfluxerr, highflux)
                        #high lim
                        beamhigh2=10**float(flux_high_freq_high_beam2[2])
                        beamhigh2=format_number(beamhigh2)

                        highflux2=10**float(flux_high_freq_high_beam2[5])
                        highfluxerr2=log_to_linear_uncertainty(flux_high_freq_high_beam2[5],flux_high_freq_high_beam2[6])

                        highflux2=round_to_sig_figs(sigsmart,highflux2, obs_sigs)
                        highfluxerr2=match_decimal_places(sigsmart,highfluxerr2, highflux2)

                        if fluxs_with_lowest_freq != 'NA':
                            flux_low_freq_low_beam,flux_low_freq_low_beam2 = jet_extrap(fluxs_with_lowest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                            #low lim
                            freqlow=10**float(flux_low_freq_low_beam[3])
                            if freqlow<10:
                                freqlow=f"{freqlow:.1f}"
                            else:
                                freqlow=f"{freqlow:.0f}"
                            beamlow=10**float(flux_low_freq_low_beam[2])
                            beamlow=format_number(beamlow)

                            lowflux=10**float(flux_low_freq_low_beam[5])
                            lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)

                            if flux_low_freq_low_beam[6] == 'NA':
                                lowfluxerr='NA'
                                lowflux = f'{lowflux}*'
                            else:
                                lowfluxerr=log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6])
                                lowfluxerr=match_decimal_places(sigsmart,lowfluxerr, lowflux)


                            #high lim
                            freqlow2=10**float(flux_low_freq_low_beam2[3])
                            if freqlow2<10:
                                freqlow2=f"{freqlow2:.1f}"
                            else:
                                freqlow2=f"{freqlow2:.0f}"
                            beamlow2=10**float(flux_low_freq_low_beam2[2])
                            beamlow2=format_number(beamlow2)

                            lowflux2=10**float(flux_low_freq_low_beam2[5])
                            lowflux2=round_to_sig_figs(sigsmart,lowflux2, obs_sigs)


                            if flux_low_freq_low_beam2[6] == 'NA':
                                lowfluxerr2='NA'
                                lowflux2= f'{lowflux2}*'
                            else:
                                lowfluxerr2=log_to_linear_uncertainty(flux_low_freq_low_beam2[5],flux_low_freq_low_beam2[6])
                                lowfluxerr2=match_decimal_places(sigsmart,lowfluxerr2, lowflux2)

                            print(f"""{isource} &
Direct Obs &
{flux_with_smallest_beam[5]} &
{sobsflux} ± {sobsfluxerr} &
{10**float(flux_with_smallest_beam[3]):.0f} &
{beamobs} &
{flux_with_smallest_beam[4]}
\\\\
{isource} &
Hot Dust &
{flux_high_freq_high_beam[7]} &
{highflux} ± {highfluxerr} &
{10**float(flux_high_freq_high_beam[3]):.0f}  &
{beamhigh} &
{flux_high_freq_high_beam[4]}
\\\\
{isource} &
Extended Jet &
{flux_low_freq_low_beam[7]} &
{lowflux} ± {lowfluxerr} &
{freqlow} &
{beamlow} &
{flux_low_freq_low_beam[4]}
\\\\
\enddata
\\tablecomments{{
\\textbf{{Direct Observation:}} The observation with the smallest beam size between 200--400~GHz.\\\\
\\textbf{{High Dust Extrapolation:}} All observations in Table~\\ref{{tab:mr1}} at frequencies 10\\% higher than the direct observation are considered. Each flux value is extrapolated to {10**float(flux_with_smallest_beam[3]):.0f}~GHz using a power-law spectral index of $+3$, and scaled by the squared ratio of beam sizes (beam size is defined as the geometric mean of the major and minor FWHM axes). The lowest extrapolated value is reported.\\\\
\\textbf{{Low Jet Extrapolation:}} All observations in Table~\\ref{{tab:mr1}} at frequencies 10\\% lower than the direct observation are considered. Each flux value is extrapolated to {10**float(flux_with_smallest_beam[3]):.0f}~GHz using a power-law spectral index of $-0.5$; no beam-size corrections are applied. The lowest extrapolated value is reported.
}}
\end{{deluxetable*}}""")
                        else:
                            print(f"""{isource} &
Direct Obs&
{flux_with_smallest_beam[5]} &
{sobsflux} ± {sobsfluxerr} &
{10**float(flux_with_smallest_beam[3]):.0f} &
{beamobs} &
{flux_with_smallest_beam[4]}
\\\\
{isource} &
Hot Dust &
{flux_high_freq_high_beam[7]} &&
{highflux} ± {highfluxerr} &
{10**float(flux_high_freq_high_beam[3]):.0f}  &
{beamhigh} &
{flux_high_freq_high_beam[4]}
\\\\
\enddata
\\tablecomments{{
\\textbf{{Direct Observation:}} The observation with the smallest beam size between 200--400~GHz.\\\\
\\textbf{{High Dust Extrapolation:}} All observations in Table~\\ref{{tab:mr1}} at frequencies 10\\% higher than the direct observation are considered. Each flux value is extrapolated to {10**float(flux_with_smallest_beam[3]):.0f}~GHz using a power-law spectral index of $+3$, and scaled by the squared ratio of beam sizes (assuming beam size is defined as the geometric mean of the major and minor FWHM axes). The lowest extrapolated value is reported.\\\\
\\textbf{{Low Jet Extrapolation:}} Not applicable for this source, as no observations 10\\% below the frequency of the direct observation are available.
}}
        \end{{deluxetable*}}""")
                    else:
                        flux_low_freq_low_beam,flux_low_freq_low_beam2 = jet_extrap(fluxs_with_lowest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                    
                        if flux_low_freq_low_beam[6]!='NA':
                            print_low=f'{10**float(flux_low_freq_low_beam[5]):.2f} ± {log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6]):.2f}'
                        else:
                            print_low = f'{10**float(flux_low_freq_low_beam[5]):.2f}*'
                        if fluxs_with_lowest_freq != 'NA':


                            freqlow=10**float(flux_low_freq_low_beam[3])
                            if freqlow<10:
                                freqlow=f"{freqlow:.1f}"
                            else:
                                freqlow=f"{freqlow:.0f}"
                            beamlow=10**float(flux_low_freq_low_beam[2])
                            beamlow=format_number(beamlow)

                            if flux_low_freq_low_beam[6]=='NA':
                                lowflux=10**float(flux_low_freq_low_beam[5])
                                lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)
                                print_lowflux = f'{lowflux}*'

                            else:
                                lowflux=10**float(flux_low_freq_low_beam[5])
                                lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)

                                lowfluxerr=log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6])
                                lowfluxerr=match_decimal_places(sigsmart,lowfluxerr, lowflux)

                                print_lowflux = f'{lowflux} ± {lowfluxerr}'


                            print(f"""{isource} &
Direct Obs&
{flux_with_smallest_beam[5]} &
{sobsflux} ± {sobsfluxerr} &
{10**float(flux_with_smallest_beam[3]):.0f} &
{beamobs} &
{flux_with_smallest_beam[4]}
\\\\
{isource} &
Extended Jet &
{flux_low_freq_low_beam[7]} &
{lowflux} ± {lowfluxerr} &
{freqlow} &
{beamlow} &
{flux_low_freq_low_beam[4]} 
\\\\
\enddata
\\tablecomments{{
\\textbf{{Direct Observation:}} The observation with the smallest beam size between 200--400~GHz.\\\\
\\textbf{{High Dust Extrapolation:}} Not applicable for this source, as no observations 10\\% above the frequency of the direct observation are available.\\\\
\\textbf{{Low Jet Extrapolation:}} All observations in Table~\\ref{{tab:mr1}} at frequencies 10\\% lower than the direct observation are considered. Each flux value is extrapolated to {10**float(flux_with_smallest_beam[3]):.0f}~GHz using a power-law spectral index of $-0.5$; no beam-size corrections are applied. The lowest extrapolated value is reported.
}}    
    \end{{deluxetable*}}""")
                        else:
                            print(f"""{isource} &
    Direct Obs&
    {flux_with_smallest_beam[5]} &
    {sobsflux} ± {sobsfluxerr} &
    {10**float(flux_with_smallest_beam[3]):.0f} &
    {beamobs} &
    {flux_with_smallest_beam[4]}
    \\\\
    \enddata
\\tablecomments{{
\\textbf{{Direct Observation:}} The observation with the smallest beam size between 200--400~GHz.\\\\
\\textbf{{High Dust Extrapolation:}} Not applicable for this source, as no observations 10\\% above the frequency of the direct observation are available.\\\\
\\textbf{{Low Jet Extrapolation:}} Not applicable for this source, as no observations 10\\% below the frequency of the direct observation are available.
}}    
    \end{{deluxetable*}}""")
            else:
                log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
                log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
                bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
                if not fluxs2:
                    nograph.append([isource,bhs])
                    namesandfiles.append([isource,'chosenfile','noobs'])
                else:
                    flux_with_smallest_beam = find_flux_with_smallest_beam(fluxs2,isource)
                    namesandfiles.append([isource,'chosenfile',flux_with_smallest_beam[6]])
            if write==True:
                sys.stdout = original_stdout

In [ ]:
#Original Extrapolate High, Low, and Chosen Observations Fluxes
import matplotlib.pyplot as plt
import sys

write=False

sigsmart=True

namesandfiles=[]

#set extrap to 1 just to extrapolate values for the chosen sources
extrap=1

if extrap==0:
    write=False

def round_to_sig_figs(sigsmart,x, n):
      if sigsmart==False:
          return(x,n)
      if x == 0:
            return 0
      else:
            digits = -int(math.floor(math.log10(abs(x)))) + (n - 1)
            return round(x, digits)
      
def round_to_sig_figs2(sigsmart,x, n):
      if sigsmart==False:
          return(x,n)
      if x == 0:
            return 0
      else:
            digits = -int(math.floor(math.log10(abs(x)))) + (n - 1)
            return round(x, digits)

def count_sig_figs(num_input):
			num_str = str(num_input).strip().replace(',', '')
			# Strip sign
			if num_str.startswith(('+', '-')):
						num_str = num_str[1:]

			# Handle scientific notation strings by expanding to a plain decimal string
			if 'e' in num_str.lower():
						mant, exp = num_str.lower().split('e', 1)
						exp = int(exp)

						# Separate mantissa integer/decimal
						if '.' in mant:
									int_part, dec_part = mant.split('.', 1)
						else:
									int_part, dec_part = mant, ''

						# Digits of mantissa with no dot
						digits = ''.join(ch for ch in (int_part + dec_part) if ch.isdigit())

						# Original decimal index within the digits string
						orig_idx = len(int_part)

						# New decimal index after shifting by exponent
						new_idx = orig_idx + exp

						if new_idx <= 0:
									integer = '0'
									decimal = '0' * (-new_idx) + digits
									num_str = integer + '.' + decimal
						elif new_idx >= len(digits):
									integer = digits + '0' * (new_idx - len(digits))
									num_str = integer  # no decimal part needed
						else:
									integer = digits[:new_idx]
									decimal = digits[new_idx:]
									num_str = integer + '.' + decimal

			# Now count sig figs on the plain decimal representation
			if '.' in num_str:
						integer, decimal = num_str.split('.', 1)
						sig_part = integer.lstrip('0') + ''.join(c for c in decimal if c.isdigit())
						return sum(c.isdigit() for c in sig_part)
			else:
						sig_part = num_str.lstrip('0').rstrip('0')
						return sum(c.isdigit() for c in sig_part)


def match_decimal_places(sigsmart,target, reference):
      if sigsmart==False:
          return f"{target}"
      # Count digits after decimal in the reference (as string)
      ref_str = str(reference)
      if '.' in ref_str:
            decimals = len(ref_str.split('.')[1])
      else:
            decimals = 0
      
      # Format the target to that number of decimal places
      return f"{target:.{decimals}f}"

sourcesofinterest=('4258','1194','3079','4945','Circinus','1068')

lim1=[200,400]

threshold=0.0015

get_freq_chosen=[]

def find_flux_with_smallest_beam(arrays,isource):
    # Find the index of the smallest beam size in the array
    # flux,fluxunc,beam,freq,date,telescope
    arrays = np.array(arrays)
    index_of_smallest_beam = np.argmin(arrays[:, 2].astype(float))
    flux = arrays[index_of_smallest_beam, 0]
    fluxunc = arrays[index_of_smallest_beam, 1]
    beam = arrays[index_of_smallest_beam, 2]
    freq = arrays[index_of_smallest_beam, 3]
    date = arrays[index_of_smallest_beam, 4].replace('_1','')
    tele = arrays[index_of_smallest_beam, 5]
    file = arrays[index_of_smallest_beam, 6].replace('\n','')
    return flux, fluxunc, beam, freq, date, tele, file

def find_fluxs_with_high_freq(arrays, logfreq_flux_with_smallest_beam):
    # Find the index of the smallest beam size in the array
    freq_chosen=10**float(logfreq_flux_with_smallest_beam)

    arrays = np.array(arrays)
    index_of_highest_freq = np.argmax(arrays[:, 3].astype(float))
    high_freq=10**float(arrays[index_of_highest_freq, 3])

    if high_freq==freq_chosen:
        return('NA')
    
    result_entries = []

    #flux = arrays[index_of_highest_freq, 0]
    #fluxunc = arrays[index_of_highest_freq, 1]
    #beam = arrays[index_of_highest_freq, 2]
    #freq = arrays[index_of_highest_freq, 3]
    #date = arrays[index_of_highest_freq, 4]
    #tele = arrays[index_of_highest_freq, 5]

    #result_entries.append((flux, fluxunc, beam, freq, date, tele))

    for i, entry in enumerate(arrays):
        ifreq = 10**float(entry[3])
        #if ifreq > freq_chosen:
        if (ifreq / freq_chosen) > 1.099:
            # Frequency is more than 10% 10\\% higher than the chosen frequency
            result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], entry[5]))

    return  result_entries

def find_fluxs_with_low_freq(arrays, logfreq_flux_with_smallest_beam):
    # Find the index of the smallest beam size in the array
    freq_chosen=10**float(logfreq_flux_with_smallest_beam)

    arrays = np.array(arrays)
    #these variables should be changed to represent lowest frequency... it was copy and pasted from find_fluxs_with_high_freq
    index_of_highest_freq = np.argmin(arrays[:, 3].astype(float))
    high_freq=10**float(arrays[index_of_highest_freq, 3])

    if high_freq==freq_chosen:
        return('NA')
    
    result_entries = []

    #flux = arrays[index_of_highest_freq, 0]
    #fluxunc = arrays[index_of_highest_freq, 1]
    #beam = arrays[index_of_highest_freq, 2]
    #freq = arrays[index_of_highest_freq, 3]
    #date = arrays[index_of_highest_freq, 4]
    #tele = arrays[index_of_highest_freq, 5]
    #result_entries.append((flux, fluxunc, beam, freq, date, tele))

    for i, entry in enumerate(arrays):
        ifreq = 10**float(entry[3])
        #if ifreq < freq_chosen:
        if (ifreq / freq_chosen) < 0.901:
            # Frequency is more than 10% 10\\% lower than the chosen frequency
            result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], entry[5]))

    return  result_entries

def dust_extrap_lowest(arrays, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam):

    freq_chosen=10**float(logfreq_flux_with_smallest_beam)
    beam_chosen=10**float(logbeam_flux_with_smallest_beam)

    result_entries = []

    # Find the index of the smallest beam size in the array
    for entry in arrays:
        ilogflux = float(entry[0])
        iflux = 10**float(entry[0])
        
        ilogfluxunc = entry[1]
 
        if ilogfluxunc!='NA':
            ifluxunc = log_to_linear_uncertainty(ilogflux, float(ilogfluxunc))
        else:
            ifluxunc = 'NA'

        ibeam = 10**float(entry[2])
        ifreq = 10**float(entry[3])

        iflux_extrap = iflux * (freq_chosen/ifreq)**4.5

        #dust emission is extended... not a point source
        iflux_extrap = iflux_extrap * (beam_chosen/ibeam)**2

        if ifluxunc!='NA':
            iflux_extrap_unc = ifluxunc * (freq_chosen/ifreq)**4.5
            iflux_extrap_unc = iflux_extrap_unc * (beam_chosen/ibeam)**2
            iflux_extrap_unc = linear_to_log_uncertainty(iflux_extrap,iflux_extrap_unc)
        else:
            iflux_extrap_unc = 'NA'

        iflux_extrap= np.log10(iflux_extrap)
        result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], iflux_extrap, iflux_extrap_unc, entry[5]))

    # Find the entry with the lowest extrapolated flux
    min_flux_entry = min(result_entries, key=lambda x: x[5])  # x[5] accesses the iflux_extrap
    max_flux_entry = max(result_entries, key=lambda x: x[5])

    # Return the details from the entry with the lowest extrapolated flux
    flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele = min_flux_entry  # Unpacking the details from the entry tuple
    flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2 = max_flux_entry  # Unpacking the details from the entry tuple
    return [flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele],[flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2]


def jet_extrap(arrays, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam):

    freq_chosen=10**float(logfreq_flux_with_smallest_beam)
    beam_chosen=10**float(logbeam_flux_with_smallest_beam)

    result_entries = []

    # Find the index of the smallest beam size in the array
    for entry in arrays:
        ilogflux = float(entry[0])
        iflux = 10**float(entry[0])
        
        ilogfluxunc = entry[1]
 
        if ilogfluxunc!='NA':
            ifluxunc = log_to_linear_uncertainty(ilogflux, float(ilogfluxunc))
        else:
            ifluxunc = 'NA'

        ibeam = 10**float(entry[2])
        ifreq = 10**float(entry[3])

        iflux_extrap = iflux * (freq_chosen/ifreq)**(-0.5)

        #compact jet is a point source... no beam correction
        #iflux_extrap = iflux_extrap * (beam_chosen/ibeam)**2

        if ifluxunc!='NA':
            iflux_extrap_unc = ifluxunc * (freq_chosen/ifreq)**(-0.5)
            iflux_extrap_unc = linear_to_log_uncertainty(iflux_extrap,iflux_extrap_unc)
        else:
            iflux_extrap_unc = 'NA'

        iflux_extrap= np.log10(iflux_extrap)
        result_entries.append((entry[0], entry[1], entry[2], entry[3], entry[4], iflux_extrap, iflux_extrap_unc, entry[5]))

    # Find the entry with the lowest extrapolated flux
    min_flux_entry = min(result_entries, key=lambda x: x[5])  # x[1] accesses the iflux_extrap
    max_flux_entry = max(result_entries, key=lambda x: x[5])
    # Return the details from the entry with the lowest extrapolated flux
    flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele = min_flux_entry  # Unpacking the details from the entry tuple
    flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2 = max_flux_entry  # Unpacking the details from the entry tuple

    return [flux, fluxunc, beam, freq, date, iflux_extrap, iflux_extrap_unc, tele],[flux2, fluxunc2, beam2, freq2, date2, iflux_extrap2, iflux_extrap_unc2, tele2]


def linear_to_log_uncertainty(lin_x, lin_delta_x):
    return lin_delta_x / (lin_x * np.log(10))

def log_to_linear_uncertainty(log_x, log_x_uncertainty):
    return float(log_x_uncertainty) * 10**float(log_x) * np.log(10)

graph_array1=[]
graph_array2=[]

nograph=[]

                        #'flux'
                        #'frequency'         
                        #'beam size'
                        #'date'  

for isource, data in grouped_data.items():
    breaker=1
    if extrap==1:
        for i in sourcesofinterest:
            if i.lower() in isource.lower():
                breaker=0
        if breaker==1:
            continue

    contam_table=f'contam_{isource}.tex'

    with open(contam_table, 'w') as file:
        if write==True:
            original_stdout = sys.stdout
            sys.stdout = file

        # Main loop to read and process sources from the input file
        if os.path.exists(txtnamesandcoords):
            with open(txtnamesandcoords, 'r') as infile:
                for line in infile:

                    # Parse the line into a list of name-coordinate pairs
                    params = ast.literal_eval(line)
                    for inamecoords in params:                
                        source_name = inamecoords[0][0]
                        if not isource.lower() in source_name.lower():
                            continue
                        source=source_name
                        ra = inamecoords[1][1]
                        dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                        findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                        distanceMPc=findvals[0][0]
                        distanceMPc_uncer=findvals[0][1]
                        method=findvals[1]
                        ynfindref=findvals[2]
                        refcode=findvals[3]
                        red_depend=findvals[4]
        else:
            print(f"File {txtnamesandcoords} does not exist.")

        arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
        arcsectopc=arcsectokpc*1000

        arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
        arcsectopc_unc=arcsectokpc_error*1000

        arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

        distance='NA'
        with open(Greenepath, 'r') as infile:
            j=-1
            breaker=1
            for line in infile:
                j=j+1
                if j>20:
                    continue
                source_name=line.split(' ')[0]
                source_name=source_name.replace("−","-")
                if isource=='WISEAJ043703':
                    isource='J0437'
                if isource.lower() in source_name.lower():
                    log_BH_mass_solar=line.split(' ')[3]
                    log_BH_mass_solar_offset=line.split(' ')[5]
                    breaker=0
                if 'cgcg' in isource.lower():
                    log_BH_mass_solar=7.38
                    log_BH_mass_solar_offset=0.012
                    breaker=0
                if '4945' in isource.lower():
                    log_BH_mass_solar=6.15
                    log_BH_mass_solar_offset='NA'
                    breaker=0
            if breaker!=1:
                mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)
            else:
                print(f'{source} not found')
            

            fluxs1=[]
            fluxs2=[]
            fluxsuplim=[]
            with open(source_text, 'r') as infile:
                j=-1
                for line in infile:
                    if line=='\n':
                        continue
                    line=eval(line)
                    source=line[0]
                    if isource in source:
                        file=line[7]
                        telescope=line[1]
                        if telescope=='VLBA':
                            continue
                        date=line[4].replace('_1','')
                        snr=line[6].split('\n')[0]

                        freq=np.log10(float(line[2]))
                        flux=np.log10(float(str(line[3]).split('*')[0]))
                        checklow=len(str(line[3]).split('*'))
                        if snr!='NA':
                            linflux=float(line[3].split('*')[0])
                            linfluxunc=float(linflux)/float(snr)

                            fluxunc=linear_to_log_uncertainty(linflux,linfluxunc)
                        else:
                            fluxunc='NA'

                        if '-' not in line[5]:
                            beam=np.log10(float(line[5]))
                        else:
                            continue

                        #for extrap==0 we are looking for an obs to fix the coordinates for all the fits
                        #we look for an observations in the 200-400 GHz range that are a detection... then to 
                        #all other frequency ranges.  We look for the highest resolution that is a detection
                        if extrap==0 and checklow==1:
                            if 10**freq<lim1[1] and 10**freq>lim1[0]:
                                if date=='2021/09/03':
                                    #input(linflux)
                                    #input(linfluxunc)
                                    pass
                                fluxs1.append([flux,fluxunc,beam,freq,date,telescope,file])
                            fluxs2.append([flux,fluxunc,beam,freq,date,telescope,file])

                        #for extrap==1 we are extrapolating to the chosen frequency... which is the frequency
                        #of the obs of the highest resolution in the 200-400GHz range.  In this case we do not 
                        #care about lower limits. If the observation of highest resolution is a lower limit this
                        #is what is reported... furthermore extrapolations are upper limits so that we include 
                        #non detections for this as well
                        if extrap==1:
                            if 10**freq<lim1[1] and 10**freq>lim1[0]:
                                if date=='2021/09/03':
                                    #input(linflux)
                                    #input(linfluxunc)
                                    pass
                                fluxs1.append([flux,fluxunc,beam,freq,date,telescope,file])
                            fluxs2.append([flux,fluxunc,beam,freq,date,telescope,file])
            if fluxs1:      
                graph_array1.append(isource)
                flux_with_smallest_beam = find_flux_with_smallest_beam(fluxs1,isource)

                if flux_with_smallest_beam[1]!='NA':
                    sobsfluxerr=f'{log_to_linear_uncertainty(flux_with_smallest_beam[0],flux_with_smallest_beam[1]):.1f}'
                    if sobsfluxerr=='0.0':
                        sobsflux=f'{10**float(flux_with_smallest_beam[0]):.2f}'
                        sobsfluxerr=f'{log_to_linear_uncertainty(flux_with_smallest_beam[0],flux_with_smallest_beam[1]):.2f}'
                    else:
                        sobsflux=f'{10**float(flux_with_smallest_beam[0]):.1f}'
                else:
                    sobsfluxerr='NA'
                    sobsflux=f'{10**float(flux_with_smallest_beam[0]):.1f}'
                
                
                namesandfiles.append([isource,'chosenfile',flux_with_smallest_beam[6]])
                if extrap==0:
                    continue

                logbeam_flux_with_smallest_beam=flux_with_smallest_beam[2]
                logfreq_flux_with_smallest_beam=flux_with_smallest_beam[3]
                get_freq_chosen.append([isource,10**float(logfreq_flux_with_smallest_beam),10**float(logbeam_flux_with_smallest_beam)])

                graph_array1.append(flux_with_smallest_beam)
                log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
                log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
                if '4945' in isource:
                    log_diam_bhs_microarcsec_unc=0
                bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
                graph_array1.append(bhs)
                flux_high_freq_high_beam='NA'
                flux_low_freq_low_beam='NA'

                beamobs=10**float(flux_with_smallest_beam[2])
                beamobs=format_number(beamobs)

                beamhigh='NA'
                beamlow='NA'
                freqlow='NA'
                if fluxs2:
                    fluxs_with_highest_freq = find_fluxs_with_high_freq(fluxs2,logfreq_flux_with_smallest_beam)
                    fluxs_with_lowest_freq = find_fluxs_with_low_freq(fluxs2,logfreq_flux_with_smallest_beam)
                    

                    obsflux_str = sobsflux            # e.g. "12.30" or "0.0450"
                    # Count sig figs from the string to preserve trailing zeros:
                    obs_sigs = count_sig_figs(obsflux_str)
                    print(isource)
                    if fluxs_with_highest_freq != 'NA':
                        flux_high_freq_high_beam, flux_high_freq_high_beam2 = dust_extrap_lowest(fluxs_with_highest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                        print('high')
                        print(flux_high_freq_high_beam[0])
                        print(flux_high_freq_high_beam[1])
                        #low lim
                        beamhigh=10**float(flux_high_freq_high_beam[2])
                        beamhigh=format_number(beamhigh)

                        highflux=10**float(flux_high_freq_high_beam[5])
                        highfluxerr=log_to_linear_uncertainty(flux_high_freq_high_beam[5],flux_high_freq_high_beam[6])

                        highflux=round_to_sig_figs(sigsmart,highflux, obs_sigs)
                        highfluxerr=match_decimal_places(sigsmart,highfluxerr, highflux)
                        #high lim
                        beamhigh2=10**float(flux_high_freq_high_beam2[2])
                        beamhigh2=format_number(beamhigh2)

                        highflux2=10**float(flux_high_freq_high_beam2[5])
                        highfluxerr2=log_to_linear_uncertainty(flux_high_freq_high_beam2[5],flux_high_freq_high_beam2[6])

                        highflux2=round_to_sig_figs(sigsmart,highflux2, obs_sigs)
                        highfluxerr2=match_decimal_places(sigsmart,highfluxerr2, highflux2)

                        if fluxs_with_lowest_freq != 'NA':
                            flux_low_freq_low_beam,flux_low_freq_low_beam2 = jet_extrap(fluxs_with_lowest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                            #low lim
                            freqlow=10**float(flux_low_freq_low_beam[3])
                            if freqlow<10:
                                freqlow=f"{freqlow:.1f}"
                            else:
                                freqlow=f"{freqlow:.0f}"
                            beamlow=10**float(flux_low_freq_low_beam[2])
                            beamlow=format_number(beamlow)

                            lowflux=10**float(flux_low_freq_low_beam[5])
                            lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)

                            if flux_low_freq_low_beam[6] == 'NA':
                                lowfluxerr='NA'
                                lowflux = f'{lowflux}*'
                            else:
                                lowfluxerr=log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6])
                                lowfluxerr=match_decimal_places(sigsmart,lowfluxerr, lowflux)


                            #high lim
                            freqlow2=10**float(flux_low_freq_low_beam2[3])
                            if freqlow2<10:
                                freqlow2=f"{freqlow2:.1f}"
                            else:
                                freqlow2=f"{freqlow2:.0f}"
                            beamlow2=10**float(flux_low_freq_low_beam2[2])
                            beamlow2=format_number(beamlow2)

                            lowflux2=10**float(flux_low_freq_low_beam2[5])
                            lowflux2=round_to_sig_figs(sigsmart,lowflux2, obs_sigs)


                            if flux_low_freq_low_beam2[6] == 'NA':
                                lowfluxerr2='NA'
                                lowflux2= f'{lowflux2}*'
                            else:
                                lowfluxerr2=log_to_linear_uncertainty(flux_low_freq_low_beam2[5],flux_low_freq_low_beam2[6])
                                lowfluxerr2=match_decimal_places(sigsmart,lowfluxerr2, lowflux2)
                    else:
                        flux_low_freq_low_beam,flux_low_freq_low_beam2 = jet_extrap(fluxs_with_lowest_freq, logfreq_flux_with_smallest_beam, logbeam_flux_with_smallest_beam)
                        #print('high')
                        #print(flux_low_freq_low_beam[1])                    
                        if flux_low_freq_low_beam[6]!='NA':
                            print_low=f'{10**float(flux_low_freq_low_beam[5]):.2f} ± {log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6]):.2f}'
                        else:
                            print_low = f'{10**float(flux_low_freq_low_beam[5]):.2f}*'
                        if fluxs_with_lowest_freq != 'NA':


                            freqlow=10**float(flux_low_freq_low_beam[3])
                            if freqlow<10:
                                freqlow=f"{freqlow:.1f}"
                            else:
                                freqlow=f"{freqlow:.0f}"
                            beamlow=10**float(flux_low_freq_low_beam[2])
                            beamlow=format_number(beamlow)

                            if flux_low_freq_low_beam[6]=='NA':
                                lowflux=10**float(flux_low_freq_low_beam[5])
                                lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)
                                print_lowflux = f'{lowflux}*'

                            else:
                                lowflux=10**float(flux_low_freq_low_beam[5])
                                lowflux=round_to_sig_figs(sigsmart,lowflux, obs_sigs)

                                lowfluxerr=log_to_linear_uncertainty(flux_low_freq_low_beam[5],flux_low_freq_low_beam[6])
                                lowfluxerr=match_decimal_places(sigsmart,lowfluxerr, lowflux)

                                print_lowflux = f'{lowflux} ± {lowfluxerr}'
            else:
                log_diam_bhs_microarcsec_unc = linear_to_log_uncertainty(diam_bhs_microarcsec,diam_bhs_microarcsec_unc)
                log_diam_bhs_microarcsec = np.log10(diam_bhs_microarcsec)
                bhs=[log_diam_bhs_microarcsec,log_diam_bhs_microarcsec_unc]
                if not fluxs2:
                    nograph.append([isource,bhs])
                    namesandfiles.append([isource,'chosenfile','noobs'])
                else:
                    flux_with_smallest_beam = find_flux_with_smallest_beam(fluxs2,isource)
                    namesandfiles.append([isource,'chosenfile',flux_with_smallest_beam[6]])
            if write==True:
                sys.stdout = original_stdout
            print()

In [ ]:
#vlbi comments
comments_ngc1068='NGC~1068 exhibits multiple spatial components across GHz frequencies, typically interpreted as a central nuclear source flanked by northern and southern jet features, each separated by approximately 0.2~arcsec from the nucleus. Detection of the central component varies between studies, possibly due to intrinsic variability or resolution limitations. Extrapolated fluxes are computed by summing all resolved components and scaling to 225~GHz using a power-law index of $-0.5$.'
comments_circinus='Only a single unresolved component is detected. The extrapolated flux is calculated by applying a power-law index of $-0.5$ to scale the observed value to 349~GHz.'
comments_ngc4258='NGC~4258 exhibits two compact jet components separated by 0.002~arcseconds. The brightness of the northern component is variable, while the southern component appears stable. The extrapolated flux listed here is derived by summing the flux densities of both components and extrapolating to 225~GHz using a power-law index of $-0.5$. The central region remains unresolved and is presumed to mark the location of the SMBH.'
comments_ngc3079='NGC~3079 exhibits four compact, variable radio components (A, B, C, and D), with separations ranging from 0.005\\arcsec{} to 0.06\\arcsec{}. Maser emission suggests the SMBH lies in a quiet region between components~A and B \\citet{Kondratko2005}. Detection of individual components varies across observations depending on observing frequency and intrinsic source variability. Extrapolated fluxes are computed by summing all resolved components and applying a power-law index of $-0.5$ to scale to 225~GHz.'


In [ ]:
#Low Radio VLBI TOT

import sys
import numpy as np
from itertools import groupby
import math

from decimal import Decimal, getcontext

import re
import decimal
import numpy as np

getcontext().prec = 25

def flux_key(val):
    # Numeric types (Decimal, float, int, NumPy scalars)
    if isinstance(val, (int, float, decimal.Decimal, np.floating)):
        return float(val)
    # String case: extract all numbers, pick the larger if multiple
    if isinstance(val, str):
        nums = re.findall(r'[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?', val)
        if not nums:
            raise ValueError(f"no numeric content in {val!r}")
        return min(float(n) for n in nums)
    # Fallback: try casting
    return float(val)

write1=True
write1=False

def writesource(isource, array, comments):
    contam_table=f'contam_{isource}.tex'
    with open(contam_table, 'r') as file:
        line=file.readlines()
        found=0
        i=-1
        for iline in line:
            i=i+1
            if 'Extended Jet' in iline:
                found=1
                ifound=i
        if found==1:
            foundfluxval,foundfluxvalunc=(line[ifound+2].replace('&','')).split('±')
            foundfluxval=float(foundfluxval)
    for i in get_freq_chosen:
        if isource==i[0]:
            freq_chosen=float(i[1])
            beam_chosen=float(i[2])
    alltots=writetotsource(array,freq_chosen)

    vlbi_table2=f'VLBI_tot_{isource}.tex'
            
    min_idx = min(range(len(alltots)), key=lambda i: flux_key(alltots[i][0]))
    min_val = flux_key(alltots[min_idx][0])
    newrefs = [row[2].replace(', ', '') for row in alltots]
    newrefs=list(set(newrefs))
    newrefstring = f"\\citep{{{','.join(newrefs)}}}"

    if min_val==foundfluxval:
        input('SAME FLUXES')

    if min_val>foundfluxval:
        pass
    elif min_val<foundfluxval:
        for i, iarray in enumerate(alltots):
            if i != min_idx:
                continue   # skip everything except the min entry
            freq=float(iarray[4])
            if freq<10:
                freq=f"{freq:.1f}"
            else:
                freq=f"{freq:.0f}"

            beamsize=iarray[5]
            beamsize=format_number(beamsize)

            if iarray[1]!='NA':
                printflux=f"{iarray[0]} ± {iarray[1]}"
            else:
                printflux=f'{iarray[0]}*'
    
            newref =iarray[2].replace(', ', '')
            newrefstring=newrefstring.replace(newref,'')
            newrefstring=newrefstring.replace('{,','{')
            newrefstring=newrefstring.replace(',}','}')
            newrefstring2=f"\\citep{{{newref}}}"+"$^{1}$"
            newrefstring=newrefstring2+', ' + newrefstring



            with open(contam_table, 'r') as file:
                line=file.readlines()
                found=0
                i=-1
                for iline in line:
                    i=i+1
                    if 'Extended Jet' in iline:
                        found=1
                        ifound=i

            line[ifound+1] = 'VLBI$^{1}$ &\n'  
            line[ifound+2] = f'{printflux} &\n'
            line[ifound+3] = f'{freq} &\n'
            line[ifound+4] = f'{beamsize} &\n'
            line[ifound+5] = f'{iarray[3]}  \n'
            line[ifound+11]=line[ifound+11].replace('than the direct observation', f'than the direct observation and VLBI observations from {newrefstring} are considered.')    


            # 4) Write back (overwrite)
            contam_table_new=contam_table.replace('.tex','_new.tex')
            with open(contam_table_new, "w", encoding="utf-8") as f:
                f.writelines(line)

def writetotsource(array,freq_chosen):
    returnvals=[]

    new_array = []

    # Split the array into chunks of 7
    for i in range(0, len(array), 7):
        iarray = array[i:i+7]
        new_array.append(iarray)
    
    # Sort the array by the first 5 elements of each sub-array
    new_array.sort(key=lambda x: x[:5])

    # Group the sorted array by the first 5 elements
    grouped_refs = [list(group) for _, group in groupby(new_array, key=lambda x: x[:5])]
    for i in grouped_refs:
        totflux=[]
        totflux_err=[]
        ref=i[0][1]
        date=i[0][2]
        freq=float(i[0][3])
        beamsize=i[0][4]

        for ii in i:
            if len(str(ii[6][0]).split("*"))<2:
                totflux.append(ii[6][0])
            else:
                totflux.append(ii[6][0].split('*')[0])
            if ii[6][1]!='NA':
                totflux_err.append(ii[6][1])
            else:
                totflux_err.append('NA')
        # Add the fluxes and propagate the errors

        totflux_err = [Decimal(item) for item in totflux_err if item != 'NA']
        totflux_errcop = copy.deepcopy(totflux_err)
        if totflux_err==[]:
            totflux_err = 'NA'
        else:
            findsumlen=np.sum(np.array(totflux_err))
            findsumlen1=copy.deepcopy(findsumlen)
            findsumlen=[float(item) for item in totflux_err]
            findsumlen_decimal = [Decimal(str(x)) for x in findsumlen]
            findsumlen = sum(findsumlen_decimal)

            totflux_err=np.sqrt(np.sum(np.array(totflux_err)**2))
            scale_factor = Decimal((freq_chosen / freq) ** (-0.5))
            totflux_err = totflux_err * scale_factor

        totflux1=copy.deepcopy(totflux)
        totflux2=copy.deepcopy(totflux)
        sendbacker=0

        i=-1
        for iflux in totflux:
            i=i+1
            if '-' in str(iflux):
                sendbacker=1
                totflux1[i]=float(iflux.split('-')[0])
                totflux2[i]=float(iflux.split('-')[1])
        if sendbacker==1:
            totflux1=[float(item) for item in totflux1]
            totflux2=[float(item) for item in totflux2]

            totflux1=np.sum(totflux1)
            totflux1_sigs=count_sig_figs(totflux1)
            totflux1=totflux1 * (freq_chosen/freq)**(-0.5)

            totflux2=np.sum(totflux2)
            totflux2_sigs=count_sig_figs(totflux2)
            totflux2=totflux2 * (freq_chosen/freq)**(-0.5)


            if totflux1_sigs!=totflux2_sigs:
                #input('DIFFERENT SIG FIGS BETWEEN TWO FLUXES')
                pass

            totflux1=round_to_sig_figs(sigsmart,totflux1, totflux1_sigs)
            totflux2=round_to_sig_figs(sigsmart,totflux2, totflux2_sigs)

            if totflux1<totflux2:
                totflux2=match_decimal_places(sigsmart,totflux2, totflux1)
                totflux_err=match_decimal_places(sigsmart,totflux_err, totflux1)
            else:
                totflux1=match_decimal_places(sigsmart,totflux1, totflux2)
                totflux_err=match_decimal_places(sigsmart,totflux_err, totflux2)

            sendback=f'{totflux1} , {totflux2}'
            returnvals.append([sendback,totflux_err,ref,date,freq,beamsize])

        if sendbacker==0:
            totflux=[float(item) for item in totflux]
            totflux_decimal = [Decimal(str(x)) for x in totflux]
            totflux = sum(totflux_decimal)

            if totflux_err!='NA':
                totflux_sigs=count_sig_figs(totflux+findsumlen1)
            else:
                totflux_sigs=count_sig_figs(totflux)

            scale_factor = Decimal((freq_chosen / freq) ** (-0.5))
            totflux = totflux * scale_factor

            totflux=round_to_sig_figs(sigsmart,totflux, totflux_sigs)
            
            if totflux_err!='NA':
                totflux_err=match_decimal_places(sigsmart,totflux_err, totflux)

                if float(totflux_err) == 0:
                    totflux_sigs=totflux_sigs + 1

                    totflux = sum(totflux_decimal)
                    scale_factor = Decimal((freq_chosen / freq) ** (-0.5))
                    totflux = totflux * scale_factor

                    totflux=round_to_sig_figs(sigsmart,totflux, totflux_sigs)

                    totflux_err = [float(item) for item in totflux_errcop if item != 'NA']
                    totflux_err=np.sqrt(np.sum(np.array(totflux_err))**2)

                    totflux_err = totflux_err * (freq_chosen/freq)**(-0.5)
                    totflux_err=match_decimal_places(sigsmart,totflux_err, totflux)

            returnvals.append([totflux,totflux_err,ref,date,freq,beamsize])

    return(returnvals)

lowradio=[]
#Source, Frequency [GHz], Flux [mJy], Beam size, snr, Reference

########
#NGC 3079
ngc3079=[]
########

#(B-D is 0.06", A-B is 0.02", A-C is 5 0.005"
#A is between D and B
#C between A and B
comments1=comments_ngc3079

###
#Irwin, 1988, APJ, 335: 658
###

#frequency: 5GHz
#telescope: US VLBI network in Mark 3
#date: 1986, June
#beam size: .00224 x 0.00074 arcsec
beamsize=np.sqrt(.00224*0.00074)

#Componet A
#flux: 12.3 ± 4.5 mJy
ngc3079.append('NGC3079')
ngc3079.append('Irwin, 1988')
ngc3079.append('1986, June')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet A')
ngc3079.append(('12.3','4.5'))

#Componet B
#flux: 12.1 ± 6.7 mJy
ngc3079.append('NGC3079')
ngc3079.append('Irwin, 1988')
ngc3079.append('1986, June')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('12.1','6.7'))

#Componet C
#flux: 19.1 ± 27.0 mJy
ngc3079.append('NGC3079')
ngc3079.append('Irwin, 1988')
ngc3079.append('1986, June')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet C')
ngc3079.append(('19.1','27.0'))

###
#Trotter, 1998, APJ 495: 740
###

#frequency: 5 GHz
#telescope: NRAO VLBA
#date: 1992 September 15  
#beam size: .0084 x .0025 "
beamsize=np.sqrt(.0084*.0025)

#Componet A
#flux: 6.0 ± 0.5 mJy
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1992, September 15')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet A')
ngc3079.append(('6.0','0.5'))

#Componet B
#flux: 16.8 ± 0.4 mJy
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1992, September 15')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('16.8','0.4'))

#Componet D
#flux: 4.3 ± 0.5 mJy
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1992, September 15')
ngc3079.append('5')
ngc3079.append(beamsize)
ngc3079.append('Componet D')
ngc3079.append(('4.3','0.5'))

#frequency: 8 GHz
#telescope: NRAO VLBA
#date: 1992 September 29
#beam size: .0018 x .0006 "
beamsize=np.sqrt(.0018*.0006)

#Componet A
#flux: 37 ± 2
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1992, September 29')
ngc3079.append('8')
ngc3079.append(beamsize)
ngc3079.append('Componet A')
ngc3079.append(('37','2'))

#Componet B
#flux: 15 ± 2
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1992, September 29')
ngc3079.append('8')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('15','2'))

#frequency: 22
#telescope: NRAO VLBA
#date: 1995 January 9
#beam size: .0015 x .00098"
beamsize=np.sqrt(.0015*.00098)

#Componet A
#flux: 6 ± 1
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1995, January 9')
ngc3079.append('22')
ngc3079.append(beamsize)
ngc3079.append('Componet A')
ngc3079.append(('6','1'))

#Componet B
#flux: 16 ± 1
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1995, January 9')
ngc3079.append('22')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('16','1'))

#Componet C
#flux: 11 ± 2
ngc3079.append('NGC3079')
ngc3079.append('Trotter, 1998')
ngc3079.append('1995, January 9')
ngc3079.append('22')
ngc3079.append(beamsize)
ngc3079.append('Componet C')
ngc3079.append(('11','2'))

###
#Sawada-Satoh, 2000, PASJ 52: 421
###

#VLBI: Very Long Baseline Array (VLBA), phased VLA (Y) and the Effelsberg (EB) 100-m
#EB: 8.4, 15, 22
#VLA: 1.4, 22
#VLBA: All

#frequency: 1.4 GHz
#telescope: VLBA, VLA
#date: 1996 October 20
#beam size: .0147 x .0128 "
beamsize=np.sqrt(.0147*.0128)

#Componet A+C
#flux: 5.9 ± 1.7
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('1.4')
ngc3079.append(beamsize)
ngc3079.append('Componet A+C')
ngc3079.append(('5.9','1.7'))

#Componet B
#flux: 5.7 ± 1.7
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('1.4')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('5.7','1.7'))

#frequency: 8.4 GHz
#telescope: VLBA, EB
#date: 1996 October 20
#beam size: 0.0013 x 0.00091 "
beamsize=np.sqrt(0.0013*0.0009)

#Componet A
#flux: 1.9 ± 0.6
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('8.4')
ngc3079.append(beamsize)
ngc3079.append('Componet A')
ngc3079.append(('1.9', '0.6'))

#Componet B
#flux: 14.2 ± 0.7
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('8.4')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('14.2', '0.7'))

#Componet C
#flux: 2.1 ± 0.9
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('8.4')
ngc3079.append(beamsize)
ngc3079.append('Componet C')
ngc3079.append(('2.1', '0.9'))

#frequency: 15 GHz
#telescope: VLBA, EB
#date: 1996 October 20
#beam size: 0.00066 x 0.00057 "
beamsize=np.sqrt(0.00066*0.00057)

#Componet B
#flux: 24.4 ± 0.6
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('15')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('24.4', '0.6'))

#frequency: 22 GHz
#telescope: VLBA, EB, VLA
#date: 1996 October 20
#beam size: 0.00041 x 0.0003 "
beamsize=np.sqrt(0.00041*0.0003)

#Componet B
#flux: 12.1 ± 0.7
ngc3079.append('NGC3079')
ngc3079.append('Sawada, 2000')
ngc3079.append('1996/10/20')
ngc3079.append('22')
ngc3079.append(beamsize)
ngc3079.append('Componet B')
ngc3079.append(('12.2', '0.7'))

writesource('NGC3079',ngc3079,comments1)

#########
#NGC 1068
ngc1068=[]
#########

comments1=comments_ngc1068

###
#Ulvestad, 1987, Astronomical Journal, 93:22, 10.1086/114286:
###

#North Center and South Components Seperated by 0.3"

#frequency: 1.4 GHz
#telescope: VLBI using the European VLBI Network (EVN)
#date: 1984 June 16
#beam size: 0.06" X 0.04"
beamsize=np.sqrt(0.06*0.04)
#Uncertainties of 10%
#Unresolved Core:
#Total: 89 + 10 mJy (71 + 10 mJy in a component less than 0.02" ( 1.5 pc) in diameter)

#North
#23
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1984, June 16')
ngc1068.append('1.4')
ngc1068.append(beamsize)
ngc1068.append('North')
ngc1068.append(('23', f'{23/10:.2f}'))

#Center
#89
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1984, June 16')
ngc1068.append('1.4')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('89', f'{89/10:.2f}'))

#South
#35
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1984, June 16')
ngc1068.append('1.4')
ngc1068.append(beamsize)
ngc1068.append('South')
ngc1068.append(('35', f'{35/10:.2f}'))

#frequency: 15 GHz
#telescope: VLA
#date: 1983 Nov 3 
#beam size: 0.136" X 0.121"
beamsize=np.sqrt(0.136*0.121)
#Uncertainties of 20%

#North
#54
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('15')
ngc1068.append(beamsize)
ngc1068.append('North')
ngc1068.append(('54', f'{54/20:.2f}'))

#Center
#75
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('15')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('75', f'{75/20:.2f}'))

#South
#29
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('15')
ngc1068.append(beamsize)
ngc1068.append('South')
ngc1068.append(('29', f'{29/20:.2f}'))

#frequency: 23 GHz
#telescope: VLA
#date: 1983 Nov 2 
#beam size: 0.075" X 0.074"
beamsize=np.sqrt(0.075*0.074)
#Uncertainties of 40%

#North
#33
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('23')
ngc1068.append(beamsize)
ngc1068.append('North')
ngc1068.append(('33', f'{33/40:.2f}'))

#Center
#63
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('23')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('63', f'{63/40:.2f}'))

#South
#24
ngc1068.append('NGC1068')
ngc1068.append('Ulvestad, 1987')
ngc1068.append('1983, Nov 3')
ngc1068.append('23')
ngc1068.append(beamsize)
ngc1068.append('South')
ngc1068.append(('24', f'{24/40:.2f}'))

###
#Greenhill, 1996, APJ 472: L21
###

#frequency: 22 GHz
#date: 1994 Nov 5
#telescope: VLBA and VLA
#beam size: 0.0014 x .001 mas
beamsize=np.sqrt(0.0014*0.001)
#Core

#<0.45mJy at 1 sigma => 1.35 mJy -> 1.4 mJy
ngc1068.append('NGC1068')
ngc1068.append('Greenhill, 1996')
ngc1068.append('1994, Nov 5')
ngc1068.append('22')
ngc1068.append(beamsize)
ngc1068.append('Core')
ngc1068.append(('1.4*', 'NA'))

###
#Krips, 2006, A&A: 446:113
###

#frequency: 100GHz
#telescope: IRAM PdBI
#date: 2003 February
#beam: 2.1 x 1.2"
beamsize=np.sqrt(2.1*1.2)

#North
#17 ± 0.5
ngc1068.append('NGC1068')
ngc1068.append('Krips, 2006')
ngc1068.append('2003, February')
ngc1068.append('100')
ngc1068.append(beamsize)
ngc1068.append('North')
ngc1068.append(('17', '0.5'))

#Center
#36 ± 0.4 mJy
ngc1068.append('NGC1068')
ngc1068.append('Krips, 2006')
ngc1068.append('2003, February')
ngc1068.append('100')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('36', '0.4'))

#South
#7 ±  0.6
ngc1068.append('NGC1068')
ngc1068.append('Krips, 2006')
ngc1068.append('2003, February')
ngc1068.append('100')
ngc1068.append(beamsize)
ngc1068.append('South')
ngc1068.append(('7', '0.6'))

#frequency: 300GHz
#telescope: IRAM PdBI
#date: February 2003
#beam: 1.0 x 0.6"
beamsize=np.sqrt(1*0.6)

#North
#6 ± 1.0
ngc1068.append('NGC1068')
ngc1068.append('Krips, 2006')
ngc1068.append('2003, February')
ngc1068.append('300')
ngc1068.append(beamsize)
ngc1068.append('North')
ngc1068.append(('6', '1'))

#Center
#22 ± 0.8
ngc1068.append('NGC1068')
ngc1068.append('Krips, 2006')
ngc1068.append('2003, February')
ngc1068.append('300')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('22', '0.8'))

#frequency: 20GHz
#telescope: VLBA, GBT, VLA
#date: 2021/11/14
#beam: 1.34 × 0.52 mas,
beamsize=np.sqrt(.00134*0.00052)

ngc1068.append('NGC1068')
ngc1068.append('Gallimore, 2024')
ngc1068.append('2021/11/14')
ngc1068.append('22')
ngc1068.append(beamsize)
ngc1068.append('Center')
ngc1068.append(('0.4515', '0.0001'))

writesource('NGC1068', ngc1068, comments1)

#########
#Circinus
circinus=[]
#########

comments1=comments_circinus

###
#Elmouttie, 1998, MNRAS 297: 1202
###

#frequency: 1.37 GHz
#Telescobe :ATCA in Narrabi
#Date: March 1994 - December 1996
#beam size: 6.0 x 5.4 "
beamsize=np.sqrt(6*5.4)

#flux: 120 ± 0.110 mJy
circinus.append('circinus')
circinus.append('Elmouttie, 1998')
circinus.append('1994/03-1996/12')
circinus.append('1.4')
circinus.append(beamsize)
circinus.append('NA')
circinus.append(('120', '12'))

#frequency: 2.37 GHz
#Telescobe :ATCA in Narrabi
#Date: March 1994 - December 1996
#beam size: 3.4 x 3.1 "
beamsize=np.sqrt(3.4*3.1)
#flux: 70 ± 0.060 mJy
circinus.append('circinus')
circinus.append('Elmouttie, 1998')
circinus.append('1994/03-1996/12')
circinus.append('2.37')
circinus.append(beamsize)
circinus.append('NA')
circinus.append(('70', '7'))

#frequency: 4.80 GHz
#Telescobe :ATCA in Narrabi
#Date: March 1994 - December 1996
#beam size: 1.4 x 1.3 "
beamsize=np.sqrt(1.4*1.3)
#flux: 50 ± 0.080 mJy
circinus.append('circinus')
circinus.append('Elmouttie, 1998')
circinus.append('1994/03-1996/12')
circinus.append('4.8')
circinus.append(beamsize)
circinus.append('NA')
circinus.append(('50', '5'))

#frequency: 8.64 GHz
#Telescobe :ATCA in Narrabi
#Date: March 1994 - December 1996
#beam size: 0.9 x 0.8 "
beamsize=np.sqrt(0.9*0.8)
#flux: 50 ± 0.095 mJy
circinus.append('circinus')
circinus.append('Elmouttie, 1998')
circinus.append('1994/03-1996/12')
circinus.append('8.64')
circinus.append(beamsize)
circinus.append('NA')
circinus.append(('50', '5'))

writesource('Circinus',circinus,comments1)

#########
#NGC 4258
ngc4258=[]
#########

#North and South Componets Seperated by 0.002"
comments1=comments_ngc4258

###
#Herrnstein 1998 ApJ 497 69 
    #DOI 10.1086/31128
###

#frequency:22 GHz
#telescope: VLBA and VLA
#date: 1997 March 7, 23 and April 7
#beam size: 0.0006 x 0.0003 "
beamsize=np.sqrt(0.0006*0.0003)

#North
#2.5 - 3.5 ± 0.3 mJy
ngc4258.append('ngc4258')
ngc4258.append('Herrnstein, 1998b')
ngc4258.append('1997, March 7, 23 and April 7')
ngc4258.append('22')
ngc4258.append(beamsize)
ngc4258.append('North')
ngc4258.append((f'2.5-3.5','0.03'))

#Center
#<0.220 mJy
ngc4258.append('ngc4258')
ngc4258.append('Herrnstein, 1998b')
ngc4258.append('1997, March 7, 23 and April 7')
ngc4258.append('22')
ngc4258.append(beamsize)
ngc4258.append('Center')
ngc4258.append((f'0.220*', 'NA'))
#South
#0.5 ± 0.1 mJy
ngc4258.append('ngc4258')
ngc4258.append('Herrnstein, 1998b')
ngc4258.append('1997, March 7, 23 and April 7')
ngc4258.append('22')
ngc4258.append(beamsize)
ngc4258.append('South')
ngc4258.append(('0.5', '0.1'))

writesource('NGC4258', ngc4258, comments1)

In [ ]:
#Low Radio VLBI INDIV

'''

import sys
import numpy as np
from itertools import groupby
import math

from decimal import Decimal, getcontext

getcontext().prec = 25

write1=False

def writesource(isource, array, comments):
    for i in get_freq_chosen:
        if isource==i[0]:
            freq_chosen=float(i[1])
            beam_chosen=float(i[2])
    alltots=writetotsource(array,freq_chosen)

    vlbi_table2=f'VLBI_tot_{isource}.tex'

    with open(vlbi_table2, 'w') as file:
        if write1==True:
            original_stdout = sys.stdout
            sys.stdout = file

        print(fr"""\begin{{deluxetable*}}{{lllllll}}\
\tablecaption{{VLBI extrapolations for {isource} to {freq_chosen:.0f} GHz}}
\tablehead{{ 
\colhead{{Source}} & \colhead{{Extrap Flux[mJy]$}} & \colhead{{Freq[GHz]}} & \colhead{{beamsize["]}} & \colhead{{Reference}} & \colhead{{Date}} 
}}
\startdata""")
        
        for iarray in alltots:
            freq=float(iarray[4])
            if freq<10:
                freq=f"{freq:.1f}"
            else:
                freq=f"{freq:.0f}"

            beamsize=iarray[5]
            beamsize=format_number(beamsize)

            if iarray[1]!='NA':
                printflux=f"{iarray[0]} ± {iarray[1]}"
            else:
                printflux=f'{iarray[0]}*'
    
            newref = iarray[2].replace(", ", "")

            print(f"""{isource} &
{printflux} &
{freq} &
{beamsize} &
{newref} &
{iarray[3]} 
\\\\""")
            
        print(fr"""\enddata
\tablecomments{{{comments}}}
\end{{deluxetable*}}""")

        if write1==True:
            sys.stdout = original_stdout
'''

In [ ]:
#check for sources without data at 200-400 GHz
print(nograph)

In [ ]:
import numpy as np
from astropy import constants as const
from astropy import units as u

def modified_blackbody_shape(nu, T, beta):
    """
    Modified blackbody shape using Astropy constants:
        S(nu) ∝ nu^(beta+3) / (exp(h nu / k_B T) - 1)
    Parameters
    ----------
    nu : Quantity
        Frequency (with units)
    T : Quantity
        Temperature (with units)
    beta : float
        Emissivity index
    """
    x = (const.h * nu / (const.k_B * T)).decompose().value
    return (nu.to(u.Hz).value ** (beta + 3)) / (np.exp(x) - 1)

# Example: compare 20 K and 1500 K at 200 GHz
nu = 200 * u.GHz
beta = 1.5

F_20   = modified_blackbody_shape(nu, 20 * u.K, beta)
F_1500 = modified_blackbody_shape(nu, 2000 * u.K, beta)
ratio  = F_1500 / F_20

print(f"nu = {nu}")
print(f"S(nu) shape at T=20 K   : {F_20:.3e}")
print(f"S(nu) shape at T=1500 K : {F_1500:.3e}")
print(f"Ratio (1500 K / 20 K)   : {ratio:.2f}")
